In [ ]:
#Installing necessary packages
!pip install geopandas

#Installing needed geo based libraries
!pip install libpysal
!pip install splot
!apt update
!apt upgrade
!apt install gdal-bin python-gdal python3-gdal 

# Install rtree - Geopandas requirment
!apt install python3-rtree 
!pip install pysal

In [ ]:
!pip install fbprophet

In [ ]:
##Importing needed libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import math
import csv
import geopandas as gpd
import matplotlib.pyplot as plt
from libpysal.weights.contiguity import Queen
from libpysal import examples
import numpy as np
import pandas as pd
import geopandas as gpd
import os
import splot
import pysal
from libpysal import weights

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
#Loading demographic data 
demo_data_2018 = pd.read_csv('acs_ct_2018.csv')
demo_data_2018['year'] = '2018'
demo_data_2017 = pd.read_csv('acs_ct_2017.csv')
demo_data_2017['year'] = '2017'
demo_data_2016 = pd.read_csv('acs_ct_2016.csv')
demo_data_2016['year'] = '2016'
demo_data_2015 = pd.read_csv('acs_ct_2015.csv')
demo_data_2015['year'] = '2015'
demo_data_2014 = pd.read_csv('acs_ct_2014.csv')
demo_data_2014['year'] = '2014'
demo_data_2013 = pd.read_csv('acs_ct_2013.csv')
demo_data_2013['year'] = '2013'
demo_data_2012 = pd.read_csv('acs_ct_2012.csv')
demo_data_2012['year'] = '2012'
demo_data_2011 = pd.read_csv('acs_ct_2011.csv')
demo_data_2011['year'] = '2011'
demo_data_2010 = pd.read_csv('acs_ct_2010.csv')
demo_data_2010['year'] = '2010'

In [ ]:
#Loading social vulnerability data
svi_2018 = pd.read_csv('NewYork_2018.csv')
svi_2018['year'] = '2018'
svi_2016 = pd.read_csv('NewYork_2016.csv')
svi_2016['year'] = '2016'
svi_2014 = pd.read_csv('NewYork_2014.csv')
svi_2014['year'] = '2014'
svi_2010 = pd.read_csv('NewYork_2010.csv')
svi_2010['year'] = '2010'
svi_2000 = pd.read_csv('NewYork_2000.csv')
svi_2000['year'] = '2000'

In [ ]:
#Defining logic to calculate hellinger distance - probability distribution of a community's SES wrt ideal conditions
def calc_dist_from_ideal(row,div_int):

#div_int is the number of classes within each probability distribution
  dist_array = np.array(row)
  
  #Assigning probability = 1 for all ideal conditions
  observation = np.cumsum(dist_array/max(np.sum(dist_array),1))

  if np.sum(dist_array) > 0:
    anc = [1]*len(dist_array)
  else:
    anc = [0]*len(dist_array)
  
  #Calculating probability distribution of ideal condition for each of the classes uniformly
  reference = np.cumsum([h / div_int for h in anc])

  return(calc_hellinger_dist(reference, observation))

In [ ]:
#Calculating Hellinger Distance
def calc_hellinger_dist(p,q):

  import numpy as np
  from scipy.linalg import norm
  from scipy.spatial.distance import euclidean


  _SQRT2 = np.sqrt(2)     # sqrt(2) with default precision np.float64
  return np.sqrt(np.sum((np.sqrt(p) - np.sqrt(q)) ** 2)) / _SQRT2

In [ ]:
redhook = pd.read_csv('/content/redhook_census_tracts.csv')
redhook.head()

,Unnamed: 0,borough,block,lot,cd,ct2010,cb2010,schooldist,council,zipcode,firecomp,policeprct,healtharea,sanitboro,sanitsub,address,zonedist1,bldgclass,landuse,easements,ownertype,ownername,lotarea,bldgarea,comarea,resarea,officearea,retailarea,garagearea,strgearea,factryarea,otherarea,areasource,numbldgs,numfloors,unitsres,unitstotal,lotfront,lotdepth,bldgfront,bldgdepth,ext,proxcode,irrlotcode,lottype,bsmtcode,assessland,assesstot,exempttot,yearbuilt,yearalter1,yearalter2,builtfar,residfar,commfar,facilfar,borocode,bbl,condono,tract2010,xcoord,ycoord,latitude,longitude,zonemap,sanborn,taxmap,GEOID
0,0,BK,574,32,306,53,1010,15,38,11231,L101,76,4100,3,1D,202 CONOVER STREET,M2-1,K2,5.0,0,NaN,202 CONOVER STREET LLC,2500,4314,4314.0,0.0,0.0,4314.0,0.0,0.0,0.0,0.0,2,0,2.0,0,1,25.00,100.00,25.0,100.0,NaN,1,N,3,0,168750,237600,0,0,2018,0,1.73,0.0,2,0.0,3,3005740032,NaN,53,980120,186237,40.677853,-74.014890,16a,301 002,30207,36047005300
1,1,BK,340,25,306,53,1000,15,39,11231,E202,76,4100,3,2C,UNION STREET,M2-1,V1,11.0,0,C,PORT AUTHORITY OF NY AND NJ,2400,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,0,0.0,0,0,18.75,100.00,0.0,0.0,NaN,0,Y,5,5,75150,75150,75150,0,0,0,0.00,0.0,2,0.0,3,3003400025,NaN,53,982598,188825,40.684958,-74.005957,16a,301 006,30202,36047005300
2,2,BK,346,6,306,53,1000,15,39,11231,E202,76,4100,3,2C,55 HAMILTON AVENUE,M2-1,V9,11.0,0,C,PORT AUTHORITY OF NY AND NJ,1196,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,0,0.0,0,0,20.00,67.75,0.0,0.0,NaN,0,Y,5,5,65250,65250,65250,0,0,0,0.00,0.0,2,0.0,3,3003460006,NaN,53,982564,188477,40.684002,-74.006079,16a,301 006,30202,36047005300
3,3,BK,541,16,306,53,1024,15,38,11231,L131,76,4100,3,1D,MILL STREET,M1-1,Z9,NaN,1,NaN,581 CLINTON STREET,1786,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,0,0.0,0,0,57.00,67.17,0.0,0.0,N,0,Y,3,0,18450,26100,0,0,0,0,0.00,0.0,1,2.4,3,3005410016,NaN,53,983977,185211,40.675038,-74.000984,16c,301 037,30206,36047005300
4,4,BK,340,19,306,53,1000,15,39,11231,E202,76,4100,3,2C,22 UNION STREET,M2-1,V1,11.0,0,C,PORT AUTHORITY OF NY AND NJ,2500,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,0,0.0,0,0,25.00,100.00,0.0,0.0,NaN,0,Y,5,5,67950,67950,67950,0,0,0,0.00,0.0,2,0.0,3,3003400019,NaN,53,982491,188874,40.685092,-74.006342,16a,301 006,30202,36047005300


In [ ]:
#Adjusting for inflation
demo_data_2010['median_income'] = demo_data_2010['median_income']*1.15
demo_data_2010['median_rent'] = demo_data_2010['median_rent']*1.15
demo_data_2010['owner_occupied_housing_units_median_value'] = demo_data_2010['owner_occupied_housing_units_median_value']*1.15

demo_data_2011['median_income'] = demo_data_2011['median_income']*1.12
demo_data_2011['median_rent'] = demo_data_2011['median_rent']*1.12
demo_data_2011['owner_occupied_housing_units_median_value'] = demo_data_2011['owner_occupied_housing_units_median_value']*1.12

demo_data_2012['median_income'] = demo_data_2012['median_income']*1.09
demo_data_2012['median_rent'] = demo_data_2012['median_rent']*1.09
demo_data_2012['owner_occupied_housing_units_median_value'] = demo_data_2012['owner_occupied_housing_units_median_value']*1.09

demo_data_2013['median_income'] = demo_data_2013['median_income']*1.08
demo_data_2013['median_rent'] = demo_data_2013['median_rent']*1.08
demo_data_2013['owner_occupied_housing_units_median_value'] = demo_data_2013['owner_occupied_housing_units_median_value']*1.08

demo_data_2014['median_income'] = demo_data_2014['median_income']*1.06
demo_data_2014['median_rent'] = demo_data_2014['median_rent']*1.06
demo_data_2014['owner_occupied_housing_units_median_value'] = demo_data_2014['owner_occupied_housing_units_median_value']*1.06

demo_data_2015['median_income'] = demo_data_2015['median_income']*1.06
demo_data_2015['median_rent'] = demo_data_2015['median_rent']*1.06
demo_data_2015['owner_occupied_housing_units_median_value'] = demo_data_2015['owner_occupied_housing_units_median_value']*1.06

demo_data_2016['median_income'] = demo_data_2016['median_income']*1.05
demo_data_2016['median_rent'] = demo_data_2016['median_rent']*1.05
demo_data_2016['owner_occupied_housing_units_median_value'] = demo_data_2016['owner_occupied_housing_units_median_value']*1.05

demo_data_2017['median_income'] = demo_data_2017['median_income']*1.02
demo_data_2017['median_rent'] = demo_data_2017['median_rent']*1.02
demo_data_2017['owner_occupied_housing_units_median_value'] = demo_data_2017['owner_occupied_housing_units_median_value']*1.02

In [ ]:
#Concating Yearly Data to derive time-series variables
demo_data = pd.concat((demo_data_2010,demo_data_2011,demo_data_2012,demo_data_2013,demo_data_2014,demo_data_2015,demo_data_2016,demo_data_2017,demo_data_2018))
demo_data.head()

,geo_id,nonfamily_households,family_households,median_year_structure_built,rent_burden_not_computed,rent_over_50_percent,rent_40_to_50_percent,rent_35_to_40_percent,rent_30_to_35_percent,rent_25_to_30_percent,rent_20_to_25_percent,rent_15_to_20_percent,rent_10_to_15_percent,rent_under_10_percent,total_pop,male_pop,female_pop,median_age,white_pop,black_pop,asian_pop,hispanic_pop,amerindian_pop,other_race_pop,two_or_more_races_pop,not_hispanic_pop,commuters_by_public_transportation,households,median_income,income_per_capita,housing_units,vacant_housing_units,vacant_housing_units_for_rent,vacant_housing_units_for_sale,median_rent,percent_income_spent_on_rent,owner_occupied_housing_units,million_dollar_housing_units,mortgaged_housing_units,families_with_young_children,...,population_3_years_over,poverty,sales_office_employed,some_college_and_associates_degree,walked_to_work,worked_at_home,workers_16_and_over,hispanic_any_race,pop_5_years_over,speak_only_english_at_home,speak_spanish_at_home,speak_spanish_at_home_low_english,pop_15_and_over,pop_never_married,pop_now_married,pop_separated,pop_widowed,pop_divorced,do_date,armed_forces,civilian_labor_force,employed_pop,unemployed_pop,not_in_labor_force,pop_16_over,pop_in_labor_force,associates_degree,bachelors_degree,high_school_diploma,less_one_year_college,masters_degree,one_year_more_college,pop_25_years_over,commute_35_44_mins,commute_60_more_mins,commute_less_10_mins,commuters_16_over,year,male_60_to_61,male_62_to_64
0,36003940200,0,0,1983.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,NaN,0,0,0,0,0,0,0,0,0.0,0,NaN,NaN,9,9,0,0,NaN,NaN,0,0,0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20062010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010,NaN,NaN
1,10001990000,0,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,NaN,0,0,0,0,0,0,0,0,0.0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20062010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010,NaN,NaN
2,12099980400,0,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,NaN,0,0,0,0,0,0,0,0,0.0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20062010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010,NaN,NaN
3,23009990000,0,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,NaN,0,0,0,0,0,0,0,0,0.0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20062010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010,NaN,NaN
4,9003980000,0,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,NaN,0,0,0,0,0,0,0,0,0.0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20062010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010,NaN,NaN


In [ ]:
demo_data.year.value_counts()

2010    74002
2013    74001
2018    74001
2011    74001
2015    74001
2014    74001
2017    74001
2016    74001
2012    74001
Name: year, dtype: int64

In [ ]:
#Deriving race based features
demo_data['other_races'] = demo_data['black_pop']+ demo_data['asian_pop'] + demo_data['hispanic_pop'] + demo_data['amerindian_pop'] + demo_data['other_race_pop'] + demo_data['two_or_more_races_pop']

In [ ]:
demo_data.head()

,geo_id,nonfamily_households,family_households,median_year_structure_built,rent_burden_not_computed,rent_over_50_percent,rent_40_to_50_percent,rent_35_to_40_percent,rent_30_to_35_percent,rent_25_to_30_percent,rent_20_to_25_percent,rent_15_to_20_percent,rent_10_to_15_percent,rent_under_10_percent,total_pop,male_pop,female_pop,median_age,white_pop,black_pop,asian_pop,hispanic_pop,amerindian_pop,other_race_pop,two_or_more_races_pop,not_hispanic_pop,commuters_by_public_transportation,households,median_income,income_per_capita,housing_units,vacant_housing_units,vacant_housing_units_for_rent,vacant_housing_units_for_sale,median_rent,percent_income_spent_on_rent,owner_occupied_housing_units,million_dollar_housing_units,mortgaged_housing_units,families_with_young_children,...,poverty,sales_office_employed,some_college_and_associates_degree,walked_to_work,worked_at_home,workers_16_and_over,hispanic_any_race,pop_5_years_over,speak_only_english_at_home,speak_spanish_at_home,speak_spanish_at_home_low_english,pop_15_and_over,pop_never_married,pop_now_married,pop_separated,pop_widowed,pop_divorced,do_date,armed_forces,civilian_labor_force,employed_pop,unemployed_pop,not_in_labor_force,pop_16_over,pop_in_labor_force,associates_degree,bachelors_degree,high_school_diploma,less_one_year_college,masters_degree,one_year_more_college,pop_25_years_over,commute_35_44_mins,commute_60_more_mins,commute_less_10_mins,commuters_16_over,year,male_60_to_61,male_62_to_64,other_races
0,36003940200,0,0,1983.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,NaN,0,0,0,0,0,0,0,0,0.0,0,NaN,NaN,9,9,0,0,NaN,NaN,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20062010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010,NaN,NaN,0
1,10001990000,0,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,NaN,0,0,0,0,0,0,0,0,0.0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20062010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010,NaN,NaN,0
2,12099980400,0,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,NaN,0,0,0,0,0,0,0,0,0.0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20062010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010,NaN,NaN,0
3,23009990000,0,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,NaN,0,0,0,0,0,0,0,0,0.0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20062010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010,NaN,NaN,0
4,9003980000,0,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,NaN,0,0,0,0,0,0,0,0,0.0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20062010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010,NaN,NaN,0


In [ ]:
demo_data_rh = demo_data[['geo_id','total_pop','white_pop','asian_pop','black_pop','hispanic_pop','median_income','income_less_10000','income_10000_14999','income_15000_19999','income_20000_24999','income_25000_29999',
                'income_30000_34999','income_35000_39999','income_40000_44999','income_45000_49999','income_50000_59999','income_60000_74999','owner_occupied_housing_units_median_value','median_rent',
                'income_75000_99999','income_100000_124999','income_125000_149999','income_150000_199999','income_200000_or_more','high_school_diploma','less_one_year_college','one_year_more_college','bachelors_degree', 'masters_degree',
                'other_races','commuters_by_public_transportation','income_per_capita','housing_units','vacant_housing_units','year']]
demo_data_rh.head()

,geo_id,total_pop,white_pop,asian_pop,black_pop,hispanic_pop,median_income,income_less_10000,income_10000_14999,income_15000_19999,income_20000_24999,income_25000_29999,income_30000_34999,income_35000_39999,income_40000_44999,income_45000_49999,income_50000_59999,income_60000_74999,owner_occupied_housing_units_median_value,median_rent,income_75000_99999,income_100000_124999,income_125000_149999,income_150000_199999,income_200000_or_more,high_school_diploma,less_one_year_college,one_year_more_college,bachelors_degree,masters_degree,other_races,commuters_by_public_transportation,income_per_capita,housing_units,vacant_housing_units,year
0,36003940200,0,0,0,0,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,9,9,2010
1,10001990000,0,0,0,0,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,0,0,2010
2,12099980400,0,0,0,0,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,0,0,2010
3,23009990000,0,0,0,0,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,0,0,2010
4,9003980000,0,0,0,0,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,0,0,2010


In [ ]:
demo_data_rh['income_less_10000'] = (demo_data_rh['income_less_10000']/demo_data_rh['total_pop'])*100
demo_data_rh['income_10000_14999'] = (demo_data_rh['income_10000_14999']/demo_data_rh['total_pop'])*100
demo_data_rh['income_15000_19999'] = (demo_data_rh['income_15000_19999']/demo_data_rh['total_pop'])*100
demo_data_rh['income_20000_24999'] = (demo_data_rh['income_20000_24999']/demo_data_rh['total_pop'])*100
demo_data_rh['income_25000_29999'] = (demo_data_rh['income_25000_29999']/demo_data_rh['total_pop'])*100
demo_data_rh['income_30000_34999'] = (demo_data_rh['income_30000_34999']/demo_data_rh['total_pop'])*100
demo_data_rh['income_35000_39999'] = (demo_data_rh['income_35000_39999']/demo_data_rh['total_pop'])*100
demo_data_rh['income_40000_44999'] = (demo_data_rh['income_40000_44999']/demo_data_rh['total_pop'])*100
demo_data_rh['income_45000_49999'] = (demo_data_rh['income_45000_49999']/demo_data_rh['total_pop'])*100
demo_data_rh['income_50000_59999'] = (demo_data_rh['income_50000_59999']/demo_data_rh['total_pop'])*100
demo_data_rh['income_60000_74999'] = (demo_data_rh['income_60000_74999']/demo_data_rh['total_pop'])*100
demo_data_rh['income_75000_99999'] = (demo_data_rh['income_75000_99999']/demo_data_rh['total_pop'])*100
demo_data_rh['income_100000_124999'] = (demo_data_rh['income_100000_124999']/demo_data_rh['total_pop'])*100
demo_data_rh['income_125000_149999'] = (demo_data_rh['income_125000_149999']/demo_data_rh['total_pop'])*100
demo_data_rh['income_150000_199999'] = (demo_data_rh['income_150000_199999']/demo_data_rh['total_pop'])*100
demo_data_rh['income_200000_or_more'] = (demo_data_rh['income_200000_or_more']/demo_data_rh['total_pop'])*100
demo_data_rh['high_school_diploma'] = (demo_data_rh['high_school_diploma']/demo_data_rh['total_pop'])*100
demo_data_rh['less_one_year_college'] = (demo_data_rh['less_one_year_college']/demo_data_rh['total_pop'])*100
demo_data_rh['one_year_more_college'] = (demo_data_rh['one_year_more_college']/demo_data_rh['total_pop'])*100
demo_data_rh['bachelors_degree'] = (demo_data_rh['bachelors_degree']/demo_data_rh['total_pop'])*100
demo_data_rh['masters_degree'] = (demo_data_rh['masters_degree']/demo_data_rh['total_pop'])*100
demo_data_rh['other_races'] = (demo_data_rh['other_races']/demo_data_rh['total_pop'])*100
demo_data_rh['white_pop'] = (demo_data_rh['white_pop']/demo_data_rh['total_pop'])*100
demo_data_rh['black_pop'] = (demo_data_rh['black_pop']/demo_data_rh['total_pop'])*100
demo_data_rh['hispanic_pop'] = (demo_data_rh['hispanic_pop']/demo_data_rh['total_pop'])*100
demo_data_rh['asian_pop'] = (demo_data_rh['asian_pop']/demo_data_rh['total_pop'])*100

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
redhook_df = pd.DataFrame(redhook['GEOID'].unique())
redhook_df = redhook_df.rename(columns={0:'geo_id'})
redhook_df

,geo_id
0,36047005300
1,36047004700
2,36047005900
3,36047008500


In [ ]:
demo_data_rh = pd.merge(redhook_df,demo_data_rh,on='geo_id',how='left')
demo_data_rh.head()

,geo_id,total_pop,white_pop,asian_pop,black_pop,hispanic_pop,median_income,income_less_10000,income_10000_14999,income_15000_19999,income_20000_24999,income_25000_29999,income_30000_34999,income_35000_39999,income_40000_44999,income_45000_49999,income_50000_59999,income_60000_74999,owner_occupied_housing_units_median_value,median_rent,income_75000_99999,income_100000_124999,income_125000_149999,income_150000_199999,income_200000_or_more,high_school_diploma,less_one_year_college,one_year_more_college,bachelors_degree,masters_degree,other_races,commuters_by_public_transportation,income_per_capita,housing_units,vacant_housing_units,year
0,36047005300,2037,40.451645,2.994600,17.378498,38.684340,78651.95,6.136475,1.669121,0.000000,0.540010,1.865488,1.718213,0.687285,1.276387,0.343643,6.480118,2.061856,527620.0,1278.80,7.658321,1.620029,4.172803,3.632793,2.994600,NaN,NaN,NaN,NaN,NaN,59.548355,659.0,35932.0,1010,137,2010
1,36047005300,2205,38.820862,3.854875,24.308390,31.746032,83125.28,4.580499,1.541950,0.181406,2.403628,0.453515,0.952381,0.816327,0.907029,0.997732,5.623583,2.040816,505232.0,1296.96,7.029478,2.040816,2.766440,4.761905,3.673469,NaN,NaN,NaN,NaN,NaN,61.179138,622.0,38452.0,1038,139,2011
2,36047005300,2339,42.667807,2.052159,23.001283,28.815733,77100.06,3.292005,0.769560,0.384780,1.496366,0.641300,2.607952,0.000000,2.094912,0.769560,5.771697,3.505772,681250.0,1362.50,4.702864,2.565199,2.223172,4.018811,5.044891,8.123130,1.581873,10.560068,24.155622,7.353570,57.332193,617.0,41254.0,1071,138,2012
3,36047005300,2359,36.286562,2.628232,17.634591,40.398474,72418.32,2.924968,2.501060,1.102162,1.314116,0.254345,2.967359,0.000000,1.992370,1.059771,3.306486,6.104281,935172.0,1480.68,4.027130,1.526070,1.398898,4.196694,3.942348,10.428148,0.211954,7.969479,21.831284,8.478169,63.713438,661.0,36526.0,1059,148,2013
4,36047005300,2506,45.011971,3.032721,10.893855,35.395052,74981.22,3.312051,1.875499,1.237031,0.718276,0.957702,2.673583,0.199521,2.274541,2.075020,1.755786,4.429370,917748.0,1394.96,4.828412,3.551476,2.434158,3.391860,4.389465,10.933759,0.399042,7.940942,23.703113,11.532322,54.988029,643.0,39125.0,1087,82,2014


In [ ]:
#Median Income

demo_data_rh_test = demo_data_rh.copy()
demo_data_rh_test = demo_data_rh_test.rename(columns={'year': 'ds', 'median_income':'y'})

geoid = list(redhook_df['geo_id'])
median_income_2027 = []

from fbprophet import Prophet
grouped = demo_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=10,freq='Y')
    forecast = m.predict(future)
    median_income_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
# Income Less than 10000

demo_data_rh_test = demo_data_rh.copy()
demo_data_rh_test = demo_data_rh_test.rename(columns={'year': 'ds', 'income_less_10000':'y'})

geoid = list(redhook_df['geo_id'])
income_less_10000_2027 = []

from fbprophet import Prophet
grouped = demo_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=10,freq='Y')
    forecast = m.predict(future)
    income_less_10000_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
# Income Less than 10000

demo_data_rh_test = demo_data_rh.copy()
demo_data_rh_test = demo_data_rh_test.rename(columns={'year': 'ds', 'income_10000_14999':'y'})

geoid = list(redhook_df['geo_id'])
income_10000_14999_2027 = []

from fbprophet import Prophet
grouped = demo_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=10,freq='Y')
    forecast = m.predict(future)
    income_10000_14999_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
# Income Less than 10000

demo_data_rh_test = demo_data_rh.copy()
demo_data_rh_test = demo_data_rh_test.rename(columns={'year': 'ds', 'income_15000_19999':'y'})

geoid = list(redhook_df['geo_id'])
income_15000_19999_2027 = []

from fbprophet import Prophet
grouped = demo_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=10,freq='Y')
    forecast = m.predict(future)
    income_15000_19999_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
# Income Less than 10000

demo_data_rh_test = demo_data_rh.copy()
demo_data_rh_test = demo_data_rh_test.rename(columns={'year': 'ds', 'income_20000_24999':'y'})

geoid = list(redhook_df['geo_id'])
income_20000_24999_2027 = []

from fbprophet import Prophet
grouped = demo_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=10,freq='Y')
    forecast = m.predict(future)
    income_20000_24999_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
# Income Less than 10000

demo_data_rh_test = demo_data_rh.copy()
demo_data_rh_test = demo_data_rh_test.rename(columns={'year': 'ds', 'income_25000_29999':'y'})

geoid = list(redhook_df['geo_id'])
income_25000_29999_2027 = []

from fbprophet import Prophet
grouped = demo_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=10,freq='Y')
    forecast = m.predict(future)
    income_25000_29999_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
# Income Less than 10000

demo_data_rh_test = demo_data_rh.copy()
demo_data_rh_test = demo_data_rh_test.rename(columns={'year': 'ds', 'income_30000_34999':'y'})

geoid = list(redhook_df['geo_id'])
income_30000_34999_2027 = []

from fbprophet import Prophet
grouped = demo_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=10,freq='Y')
    forecast = m.predict(future)
    income_30000_34999_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
# Income Less than 10000

demo_data_rh_test = demo_data_rh.copy()
demo_data_rh_test = demo_data_rh_test.rename(columns={'year': 'ds', 'income_35000_39999':'y'})

geoid = list(redhook_df['geo_id'])
income_35000_39999_2027 = []

from fbprophet import Prophet
grouped = demo_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=10,freq='Y')
    forecast = m.predict(future)
    income_35000_39999_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
# Income Less than 10000

demo_data_rh_test = demo_data_rh.copy()
demo_data_rh_test = demo_data_rh_test.rename(columns={'year': 'ds', 'income_40000_44999':'y'})

geoid = list(redhook_df['geo_id'])
income_40000_44999_2027 = []

from fbprophet import Prophet
grouped = demo_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=10,freq='Y')
    forecast = m.predict(future)
    income_40000_44999_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
# Income Less than 10000

demo_data_rh_test = demo_data_rh.copy()
demo_data_rh_test = demo_data_rh_test.rename(columns={'year': 'ds', 'income_45000_49999':'y'})

geoid = list(redhook_df['geo_id'])
income_45000_49999_2027 = []

from fbprophet import Prophet
grouped = demo_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=10,freq='Y')
    forecast = m.predict(future)
    income_45000_49999_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
# Income Less than 10000

demo_data_rh_test = demo_data_rh.copy()
demo_data_rh_test = demo_data_rh_test.rename(columns={'year': 'ds', 'income_50000_59999':'y'})

geoid = list(redhook_df['geo_id'])
income_50000_59999_2027 = []

from fbprophet import Prophet
grouped = demo_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=10,freq='Y')
    forecast = m.predict(future)
    income_50000_59999_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
# Income Less than 10000

demo_data_rh_test = demo_data_rh.copy()
demo_data_rh_test = demo_data_rh_test.rename(columns={'year': 'ds', 'income_60000_74999':'y'})

geoid = list(redhook_df['geo_id'])
income_60000_74999_2027 = []

from fbprophet import Prophet
grouped = demo_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=10,freq='Y')
    forecast = m.predict(future)
    income_60000_74999_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
# Income Less than 10000

demo_data_rh_test = demo_data_rh.copy()
demo_data_rh_test = demo_data_rh_test.rename(columns={'year': 'ds', 'income_75000_99999':'y'})

geoid = list(redhook_df['geo_id'])
income_75000_99999_2027 = []

from fbprophet import Prophet
grouped = demo_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=10,freq='Y')
    forecast = m.predict(future)
    income_75000_99999_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
# Income Less than 10000

demo_data_rh_test = demo_data_rh.copy()
demo_data_rh_test = demo_data_rh_test.rename(columns={'year': 'ds', 'income_100000_124999':'y'})

geoid = list(redhook_df['geo_id'])
income_100000_124999_2027 = []

from fbprophet import Prophet
grouped = demo_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=10,freq='Y')
    forecast = m.predict(future)
    income_100000_124999_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
# Income Less than 10000

demo_data_rh_test = demo_data_rh.copy()
demo_data_rh_test = demo_data_rh_test.rename(columns={'year': 'ds', 'income_125000_149999':'y'})

geoid = list(redhook_df['geo_id'])
income_125000_149999_2027 = []

from fbprophet import Prophet
grouped = demo_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=10,freq='Y')
    forecast = m.predict(future)
    income_125000_149999_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
# Income Less than 10000

demo_data_rh_test = demo_data_rh.copy()
demo_data_rh_test = demo_data_rh_test.rename(columns={'year': 'ds', 'income_150000_199999':'y'})

geoid = list(redhook_df['geo_id'])
income_150000_199999_2027 = []

from fbprophet import Prophet
grouped = demo_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=10,freq='Y')
    forecast = m.predict(future)
    income_150000_199999_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
# Income Less than 10000

demo_data_rh_test = demo_data_rh.copy()
demo_data_rh_test = demo_data_rh_test.rename(columns={'year': 'ds', 'income_200000_or_more':'y'})

geoid = list(redhook_df['geo_id'])
income_200000_or_more_2027 = []

from fbprophet import Prophet
grouped = demo_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=10,freq='Y')
    forecast = m.predict(future)
    income_200000_or_more_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
# Income Less than 10000

demo_data_rh_test = demo_data_rh.copy()
demo_data_rh_test = demo_data_rh_test.rename(columns={'year': 'ds', 'high_school_diploma':'y'})

geoid = list(redhook_df['geo_id'])
high_school_diploma_2027 = []

from fbprophet import Prophet
grouped = demo_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=10,freq='Y')
    forecast = m.predict(future)
    high_school_diploma_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 4.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 4.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 4.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
# Income Less than 10000

demo_data_rh_test = demo_data_rh.copy()
demo_data_rh_test = demo_data_rh_test.rename(columns={'year': 'ds', 'less_one_year_college':'y'})

geoid = list(redhook_df['geo_id'])
less_one_year_college_2027 = []

from fbprophet import Prophet
grouped = demo_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=10,freq='Y')
    forecast = m.predict(future)
    less_one_year_college_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 4.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 4.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 4.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
# Income Less than 10000

demo_data_rh_test = demo_data_rh.copy()
demo_data_rh_test = demo_data_rh_test.rename(columns={'year': 'ds', 'one_year_more_college':'y'})

geoid = list(redhook_df['geo_id'])
one_year_more_college_2027 = []

from fbprophet import Prophet
grouped = demo_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=10,freq='Y')
    forecast = m.predict(future)
    one_year_more_college_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 4.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 4.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 4.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
# Income Less than 10000

demo_data_rh_test = demo_data_rh.copy()
demo_data_rh_test = demo_data_rh_test.rename(columns={'year': 'ds', 'bachelors_degree':'y'})

geoid = list(redhook_df['geo_id'])
bachelors_degree_2027 = []

from fbprophet import Prophet
grouped = demo_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=10,freq='Y')
    forecast = m.predict(future)
    bachelors_degree_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 4.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 4.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 4.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
# Income Less than 10000

demo_data_rh_test = demo_data_rh.copy()
demo_data_rh_test = demo_data_rh_test.rename(columns={'year': 'ds', 'masters_degree':'y'})

geoid = list(redhook_df['geo_id'])
masters_degree_2027 = []

from fbprophet import Prophet
grouped = demo_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=10,freq='Y')
    forecast = m.predict(future)
    masters_degree_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 4.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 4.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 4.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
# Income Less than 10000

demo_data_rh_test = demo_data_rh.copy()
demo_data_rh_test = demo_data_rh_test.rename(columns={'year': 'ds', 'other_races':'y'})

geoid = list(redhook_df['geo_id'])
other_races_2027 = []

from fbprophet import Prophet
grouped = demo_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=10,freq='Y')
    forecast = m.predict(future)
    other_races_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
# Income Less than 10000

demo_data_rh_test = demo_data_rh.copy()
demo_data_rh_test = demo_data_rh_test.rename(columns={'year': 'ds', 'commuters_by_public_transportation':'y'})

geoid = list(redhook_df['geo_id'])
commuters_by_public_transportation_2027 = []

from fbprophet import Prophet
grouped = demo_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=10,freq='Y')
    forecast = m.predict(future)
    commuters_by_public_transportation_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
# Income Less than 10000

demo_data_rh_test = demo_data_rh.copy()
demo_data_rh_test = demo_data_rh_test.rename(columns={'year': 'ds', 'income_per_capita':'y'})

geoid = list(redhook_df['geo_id'])
income_per_capita_2027 = []

from fbprophet import Prophet
grouped = demo_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=10,freq='Y')
    forecast = m.predict(future)
    income_per_capita_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
# Income Less than 10000

demo_data_rh_test = demo_data_rh.copy()
demo_data_rh_test = demo_data_rh_test.rename(columns={'year': 'ds', 'housing_units':'y'})

geoid = list(redhook_df['geo_id'])
housing_units_2027 = []

from fbprophet import Prophet
grouped = demo_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=10,freq='Y')
    forecast = m.predict(future)
    housing_units_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
# Income Less than 10000

demo_data_rh_test = demo_data_rh.copy()
demo_data_rh_test = demo_data_rh_test.rename(columns={'year': 'ds', 'vacant_housing_units':'y'})

geoid = list(redhook_df['geo_id'])
vacant_housing_units_2027 = []

from fbprophet import Prophet
grouped = demo_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=10,freq='Y')
    forecast = m.predict(future)
    vacant_housing_units_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
# Income Less than 10000

demo_data_rh_test = demo_data_rh.copy()
demo_data_rh_test = demo_data_rh_test.rename(columns={'year': 'ds', 'white_pop':'y'})

geoid = list(redhook_df['geo_id'])
white_pop_2027 = []

from fbprophet import Prophet
grouped = demo_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=10,freq='Y')
    forecast = m.predict(future)
    white_pop_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
# Income Less than 10000

demo_data_rh_test = demo_data_rh.copy()
demo_data_rh_test = demo_data_rh_test.rename(columns={'year': 'ds', 'black_pop':'y'})

geoid = list(redhook_df['geo_id'])
black_pop_2027 = []

from fbprophet import Prophet
grouped = demo_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=10,freq='Y')
    forecast = m.predict(future)
    black_pop_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
# Income Less than 10000

demo_data_rh_test = demo_data_rh.copy()
demo_data_rh_test = demo_data_rh_test.rename(columns={'year': 'ds', 'asian_pop':'y'})

geoid = list(redhook_df['geo_id'])
asian_pop_2027 = []

from fbprophet import Prophet
grouped = demo_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=10,freq='Y')
    forecast = m.predict(future)
    asian_pop_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
# Income Less than 10000

demo_data_rh_test = demo_data_rh.copy()
demo_data_rh_test = demo_data_rh_test.rename(columns={'year': 'ds', 'hispanic_pop':'y'})

geoid = list(redhook_df['geo_id'])
hispanic_pop_2027 = []

from fbprophet import Prophet
grouped = demo_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=10,freq='Y')
    forecast = m.predict(future)
    hispanic_pop_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
# Income Less than 10000

demo_data_rh_test = demo_data_rh.copy()
demo_data_rh_test = demo_data_rh_test.rename(columns={'year': 'ds', 'median_rent':'y'})

geoid = list(redhook_df['geo_id'])
median_rent_2027 = []

from fbprophet import Prophet
grouped = demo_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=10,freq='Y')
    forecast = m.predict(future)
    median_rent_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 6.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
demo_data_2027 = pd.DataFrame({'geo_id':geoid,'median_income':median_income_2027,'income_less_10000':income_less_10000_2027,
                               'income_10000_14999':income_10000_14999_2027,'income_15000_19999':income_15000_19999_2027,'income_20000_24999':income_20000_24999_2027,
                               'income_25000_29999':income_25000_29999_2027,'income_30000_34999':income_30000_34999_2027,'income_35000_39999':income_35000_39999_2027,
                               'income_40000_44999':income_40000_44999_2027,'income_45000_49999':income_45000_49999_2027,'income_50000_59999':income_50000_59999_2027,
                               'income_60000_74999':income_60000_74999_2027,'income_75000_99999':income_75000_99999_2027,'income_100000_124999':income_100000_124999_2027,
                               'income_125000_149999':income_125000_149999_2027,'income_150000_199999':income_150000_199999_2027,'income_200000_or_more':income_200000_or_more_2027,
                               'high_school_diploma':high_school_diploma_2027,'less_one_year_college':less_one_year_college_2027,'one_year_more_college':one_year_more_college_2027,
                               'bachelors_degree':bachelors_degree_2027,'masters_degree':masters_degree_2027,'other_races':other_races_2027,
                               'commuters_by_public_transportation':commuters_by_public_transportation_2027,'income_per_capita':income_per_capita_2027,
                               'housing_units':housing_units_2027,'vacant_housing_units':vacant_housing_units_2027,'median_rent':median_rent_2027,'white_pop':white_pop_2027,
                               'black_pop':black_pop_2027,'asian_pop':asian_pop_2027,'hispanic_pop':hispanic_pop_2027})

demo_data_2027['year'] = '2027'
demo_data_2027

,geo_id,median_income,income_less_10000,income_10000_14999,income_15000_19999,income_20000_24999,income_25000_29999,income_30000_34999,income_35000_39999,income_40000_44999,income_45000_49999,income_50000_59999,income_60000_74999,income_75000_99999,income_100000_124999,income_125000_149999,income_150000_199999,income_200000_or_more,high_school_diploma,less_one_year_college,one_year_more_college,bachelors_degree,masters_degree,other_races,commuters_by_public_transportation,income_per_capita,housing_units,vacant_housing_units,median_rent,white_pop,black_pop,asian_pop,hispanic_pop,year
0,36047005300,88237.500078,3.673657,0.545978,3.314382,-2.655295,2.404052,4.196310,0.495901,-3.414142,-1.693618,-0.285309,1.850636,10.448520,-10.991569,6.437527,4.330317,16.351430,-11.263266,2.990998,7.242160,8.607851,24.315846,43.116579,899.349115,68985.066086,1493.057694,-26.376192,1814.552769,55.304172,22.902085,-8.429385,14.969850,2027
1,36047004700,107139.158564,-2.876878,2.285901,3.749399,1.320397,-0.828882,1.572746,2.191943,0.793929,3.564199,-5.948172,1.608419,-0.551120,11.360169,0.943762,4.455201,14.034931,2.818045,6.819257,8.095929,19.845507,27.456576,37.324998,454.980078,58982.572453,1110.005323,60.361923,2488.071964,65.563607,-4.019905,0.003511,12.200261,2027
2,36047005900,94671.108845,3.538551,-10.871276,1.095397,3.143523,1.999328,5.347873,3.101730,1.425567,5.792755,-7.378379,0.753801,8.125567,14.016596,-0.374333,8.955132,3.470029,10.879021,5.451061,14.567889,48.808714,-0.267597,34.889383,611.299970,47188.241857,902.915155,-14.054450,2677.844003,64.889594,13.321007,-7.625539,18.026466,2027
3,36047008500,9236.186357,24.718412,2.611746,2.599958,3.356741,-0.995334,3.044618,1.654982,4.790598,-2.676764,0.944857,3.582363,2.120697,0.639367,-0.942782,0.247433,0.724769,2.620652,1.210775,1.741608,-3.059114,1.865711,82.803608,1501.530295,17068.899828,3207.473991,-77.730191,332.498273,2.323068,67.007571,12.222600,32.156294,2027


In [ ]:
demo_data_2017['other_races'] = demo_data_2017['black_pop']+ demo_data_2017['asian_pop'] + demo_data_2017['hispanic_pop'] + demo_data_2017['amerindian_pop'] + demo_data_2017['other_race_pop'] + demo_data_2017['two_or_more_races_pop']
demo_data_2016['other_races'] = demo_data_2016['black_pop']+ demo_data_2016['asian_pop'] + demo_data_2016['hispanic_pop'] + demo_data_2016['amerindian_pop'] + demo_data_2016['other_race_pop'] + demo_data_2016['two_or_more_races_pop']

demo_data_2017 = demo_data_2017[['geo_id','median_income','total_pop','income_less_10000','income_10000_14999','income_15000_19999','income_20000_24999','income_25000_29999',
                'income_30000_34999','income_35000_39999','income_40000_44999','income_45000_49999','income_50000_59999','income_60000_74999',
                'income_75000_99999','income_100000_124999','income_125000_149999','income_150000_199999','income_200000_or_more','high_school_diploma','less_one_year_college','one_year_more_college','bachelors_degree', 'masters_degree',
                'other_races','commuters_by_public_transportation','income_per_capita','housing_units','vacant_housing_units','white_pop','black_pop','asian_pop','hispanic_pop','year']]

demo_data_2016 = demo_data_2016[['geo_id','median_income','total_pop','income_less_10000','income_10000_14999','income_15000_19999','income_20000_24999','income_25000_29999',
                'income_30000_34999','income_35000_39999','income_40000_44999','income_45000_49999','income_50000_59999','income_60000_74999',
                'income_75000_99999','income_100000_124999','income_125000_149999','income_150000_199999','income_200000_or_more','high_school_diploma','less_one_year_college','one_year_more_college','bachelors_degree', 'masters_degree',
                'other_races','commuters_by_public_transportation','income_per_capita','housing_units','vacant_housing_units','white_pop','black_pop','asian_pop','hispanic_pop','year']]

demo_data_2017_rh = pd.merge(redhook_df,demo_data_2017,on='geo_id',how='left')
demo_data_2017_rh.head()

demo_data_2016_rh = pd.merge(redhook_df,demo_data_2016,on='geo_id',how='left')
demo_data_2016_rh.head()

,geo_id,median_income,total_pop,income_less_10000,income_10000_14999,income_15000_19999,income_20000_24999,income_25000_29999,income_30000_34999,income_35000_39999,income_40000_44999,income_45000_49999,income_50000_59999,income_60000_74999,income_75000_99999,income_100000_124999,income_125000_149999,income_150000_199999,income_200000_or_more,high_school_diploma,less_one_year_college,one_year_more_college,bachelors_degree,masters_degree,other_races,commuters_by_public_transportation,income_per_capita,housing_units,vacant_housing_units,white_pop,black_pop,asian_pop,hispanic_pop,year
0,36047005300,84656.25,2424,66,64,36,28,6,48,31,32,39,33,86,114,111,52,90,146,224,46,185,529,340,1222,593,41662.0,1084,102,1202,245,48,790,2016
1,36047004700,114187.50,2142,53,0,37,5,23,35,33,4,9,45,52,96,58,88,55,251,130,41,98,514,378,970,683,59577.0,906,62,1172,311,130,421,2016
2,36047005900,69644.40,1397,25,15,21,67,22,27,29,15,26,50,37,108,88,21,31,29,153,29,83,432,146,589,396,33886.0,676,65,808,134,18,346,2016
3,36047008500,14940.45,7281,1197,380,178,285,160,177,50,185,12,78,219,110,18,0,0,15,676,270,497,152,64,7170,1311,12254.0,3089,25,111,3269,340,3418,2016


In [ ]:
demo_data_2017_rh['income_less_10000'] = (demo_data_2017_rh['income_less_10000']/demo_data_2017_rh['total_pop'])*100
demo_data_2017_rh['income_10000_14999'] = (demo_data_2017_rh['income_10000_14999']/demo_data_2017_rh['total_pop'])*100
demo_data_2017_rh['income_15000_19999'] = (demo_data_2017_rh['income_15000_19999']/demo_data_2017_rh['total_pop'])*100
demo_data_2017_rh['income_20000_24999'] = (demo_data_2017_rh['income_20000_24999']/demo_data_2017_rh['total_pop'])*100
demo_data_2017_rh['income_25000_29999'] = (demo_data_2017_rh['income_25000_29999']/demo_data_2017_rh['total_pop'])*100
demo_data_2017_rh['income_30000_34999'] = (demo_data_2017_rh['income_30000_34999']/demo_data_2017_rh['total_pop'])*100
demo_data_2017_rh['income_35000_39999'] = (demo_data_2017_rh['income_35000_39999']/demo_data_2017_rh['total_pop'])*100
demo_data_2017_rh['income_40000_44999'] = (demo_data_2017_rh['income_40000_44999']/demo_data_2017_rh['total_pop'])*100
demo_data_2017_rh['income_45000_49999'] = (demo_data_2017_rh['income_45000_49999']/demo_data_2017_rh['total_pop'])*100
demo_data_2017_rh['income_50000_59999'] = (demo_data_2017_rh['income_50000_59999']/demo_data_2017_rh['total_pop'])*100
demo_data_2017_rh['income_60000_74999'] = (demo_data_2017_rh['income_60000_74999']/demo_data_2017_rh['total_pop'])*100
demo_data_2017_rh['income_75000_99999'] = (demo_data_2017_rh['income_75000_99999']/demo_data_2017_rh['total_pop'])*100
demo_data_2017_rh['income_100000_124999'] = (demo_data_2017_rh['income_100000_124999']/demo_data_2017_rh['total_pop'])*100
demo_data_2017_rh['income_125000_149999'] = (demo_data_2017_rh['income_125000_149999']/demo_data_2017_rh['total_pop'])*100
demo_data_2017_rh['income_150000_199999'] = (demo_data_2017_rh['income_150000_199999']/demo_data_2017_rh['total_pop'])*100
demo_data_2017_rh['income_200000_or_more'] = (demo_data_2017_rh['income_200000_or_more']/demo_data_2017_rh['total_pop'])*100
demo_data_2017_rh['high_school_diploma'] = (demo_data_2017_rh['high_school_diploma']/demo_data_2017_rh['total_pop'])*100
demo_data_2017_rh['less_one_year_college'] = (demo_data_2017_rh['less_one_year_college']/demo_data_2017_rh['total_pop'])*100
demo_data_2017_rh['one_year_more_college'] = (demo_data_2017_rh['one_year_more_college']/demo_data_2017_rh['total_pop'])*100
demo_data_2017_rh['bachelors_degree'] = (demo_data_2017_rh['bachelors_degree']/demo_data_2017_rh['total_pop'])*100
demo_data_2017_rh['masters_degree'] = (demo_data_2017_rh['masters_degree']/demo_data_2017_rh['total_pop'])*100
demo_data_2017_rh['other_races'] = (demo_data_2017_rh['other_races']/demo_data_2017_rh['total_pop'])*100
demo_data_2017_rh['white_pop'] = (demo_data_2017_rh['white_pop']/demo_data_2017_rh['total_pop'])*100
demo_data_2017_rh['black_pop'] = (demo_data_2017_rh['black_pop']/demo_data_2017_rh['total_pop'])*100
demo_data_2017_rh['hispanic_pop'] = (demo_data_2017_rh['hispanic_pop']/demo_data_2017_rh['total_pop'])*100
demo_data_2017_rh['asian_pop'] = (demo_data_2017_rh['asian_pop']/demo_data_2017_rh['total_pop'])*100

In [ ]:
demo_data_2017_rh.head()

,geo_id,median_income,total_pop,income_less_10000,income_10000_14999,income_15000_19999,income_20000_24999,income_25000_29999,income_30000_34999,income_35000_39999,income_40000_44999,income_45000_49999,income_50000_59999,income_60000_74999,income_75000_99999,income_100000_124999,income_125000_149999,income_150000_199999,income_200000_or_more,high_school_diploma,less_one_year_college,one_year_more_college,bachelors_degree,masters_degree,other_races,commuters_by_public_transportation,income_per_capita,housing_units,vacant_housing_units,white_pop,black_pop,asian_pop,hispanic_pop,year
0,36047005300,95625.00,2420,2.768595,1.570248,2.851240,1.446281,0.495868,0.661157,1.446281,0.371901,1.157025,1.694215,2.603306,4.586777,5.371901,1.652893,4.297521,6.446281,9.504132,1.900826,7.107438,22.314050,12.520661,50.082645,574,42589.0,1072,118,49.917355,9.628099,1.652893,34.545455,2017
1,36047004700,109341.96,2025,2.370370,0.000000,0.839506,0.246914,0.888889,1.975309,0.000000,0.246914,0.395062,2.024691,3.654321,5.777778,3.209877,3.358025,3.012346,11.358025,5.185185,0.987654,4.740741,23.703704,20.098765,43.555556,676,62396.0,889,92,56.444444,14.419753,6.666667,17.580247,2017
2,36047005900,72249.66,1337,0.972326,1.421092,1.495886,4.113687,0.000000,2.094241,1.421092,2.617801,0.822737,4.263276,5.310396,5.011219,5.459985,3.739716,3.365744,3.216156,10.246821,1.421092,9.274495,33.358265,11.219147,39.491399,445,40386.0,672,66,60.508601,4.487659,1.346298,27.823485,2017
3,36047008500,13471.14,7559,15.808969,6.667549,2.447414,2.275433,2.328350,2.434184,1.481677,2.407726,0.185210,0.859902,2.500331,1.084800,0.251356,0.000000,0.277815,0.185210,9.763196,2.844292,6.019315,1.772721,0.701151,98.941659,1236,11901.0,3140,26,1.058341,47.810557,5.212330,44.304802,2017


In [ ]:
demo_data_2016_rh['income_less_10000'] = (demo_data_2016_rh['income_less_10000']/demo_data_2016_rh['total_pop'])*100
demo_data_2016_rh['income_10000_14999'] = (demo_data_2016_rh['income_10000_14999']/demo_data_2016_rh['total_pop'])*100
demo_data_2016_rh['income_15000_19999'] = (demo_data_2016_rh['income_15000_19999']/demo_data_2016_rh['total_pop'])*100
demo_data_2016_rh['income_20000_24999'] = (demo_data_2016_rh['income_20000_24999']/demo_data_2016_rh['total_pop'])*100
demo_data_2016_rh['income_25000_29999'] = (demo_data_2016_rh['income_25000_29999']/demo_data_2016_rh['total_pop'])*100
demo_data_2016_rh['income_30000_34999'] = (demo_data_2016_rh['income_30000_34999']/demo_data_2016_rh['total_pop'])*100
demo_data_2016_rh['income_35000_39999'] = (demo_data_2016_rh['income_35000_39999']/demo_data_2016_rh['total_pop'])*100
demo_data_2016_rh['income_40000_44999'] = (demo_data_2016_rh['income_40000_44999']/demo_data_2016_rh['total_pop'])*100
demo_data_2016_rh['income_45000_49999'] = (demo_data_2016_rh['income_45000_49999']/demo_data_2016_rh['total_pop'])*100
demo_data_2016_rh['income_50000_59999'] = (demo_data_2016_rh['income_50000_59999']/demo_data_2016_rh['total_pop'])*100
demo_data_2016_rh['income_60000_74999'] = (demo_data_2016_rh['income_60000_74999']/demo_data_2016_rh['total_pop'])*100
demo_data_2016_rh['income_75000_99999'] = (demo_data_2016_rh['income_75000_99999']/demo_data_2016_rh['total_pop'])*100
demo_data_2016_rh['income_100000_124999'] = (demo_data_2016_rh['income_100000_124999']/demo_data_2016_rh['total_pop'])*100
demo_data_2016_rh['income_125000_149999'] = (demo_data_2016_rh['income_125000_149999']/demo_data_2016_rh['total_pop'])*100
demo_data_2016_rh['income_150000_199999'] = (demo_data_2016_rh['income_150000_199999']/demo_data_2016_rh['total_pop'])*100
demo_data_2016_rh['income_200000_or_more'] = (demo_data_2016_rh['income_200000_or_more']/demo_data_2016_rh['total_pop'])*100
demo_data_2016_rh['high_school_diploma'] = (demo_data_2016_rh['high_school_diploma']/demo_data_2016_rh['total_pop'])*100
demo_data_2016_rh['less_one_year_college'] = (demo_data_2016_rh['less_one_year_college']/demo_data_2016_rh['total_pop'])*100
demo_data_2016_rh['one_year_more_college'] = (demo_data_2016_rh['one_year_more_college']/demo_data_2016_rh['total_pop'])*100
demo_data_2016_rh['bachelors_degree'] = (demo_data_2016_rh['bachelors_degree']/demo_data_2016_rh['total_pop'])*100
demo_data_2016_rh['masters_degree'] = (demo_data_2016_rh['masters_degree']/demo_data_2016_rh['total_pop'])*100
demo_data_2016_rh['other_races'] = (demo_data_2016_rh['other_races']/demo_data_2016_rh['total_pop'])*100
demo_data_2016_rh['white_pop'] = (demo_data_2016_rh['white_pop']/demo_data_2016_rh['total_pop'])*100
demo_data_2016_rh['black_pop'] = (demo_data_2016_rh['black_pop']/demo_data_2016_rh['total_pop'])*100
demo_data_2016_rh['hispanic_pop'] = (demo_data_2016_rh['hispanic_pop']/demo_data_2016_rh['total_pop'])*100
demo_data_2016_rh['asian_pop'] = (demo_data_2016_rh['asian_pop']/demo_data_2016_rh['total_pop'])*100

In [ ]:
demo_data_2016_rh.head()

,geo_id,median_income,total_pop,income_less_10000,income_10000_14999,income_15000_19999,income_20000_24999,income_25000_29999,income_30000_34999,income_35000_39999,income_40000_44999,income_45000_49999,income_50000_59999,income_60000_74999,income_75000_99999,income_100000_124999,income_125000_149999,income_150000_199999,income_200000_or_more,high_school_diploma,less_one_year_college,one_year_more_college,bachelors_degree,masters_degree,other_races,commuters_by_public_transportation,income_per_capita,housing_units,vacant_housing_units,white_pop,black_pop,asian_pop,hispanic_pop,year
0,36047005300,84656.25,2424,2.722772,2.640264,1.485149,1.155116,0.247525,1.980198,1.278878,1.320132,1.608911,1.361386,3.547855,4.702970,4.579208,2.145215,3.712871,6.023102,9.240924,1.897690,7.632013,21.823432,14.026403,50.412541,593,41662.0,1084,102,49.587459,10.107261,1.980198,32.590759,2016
1,36047004700,114187.50,2142,2.474323,0.000000,1.727358,0.233427,1.073763,1.633987,1.540616,0.186741,0.420168,2.100840,2.427638,4.481793,2.707750,4.108310,2.567694,11.718021,6.069094,1.914099,4.575163,23.996265,17.647059,45.284781,683,59577.0,906,62,54.715219,14.519141,6.069094,19.654528,2016
2,36047005900,69644.40,1397,1.789549,1.073729,1.503221,4.795991,1.574803,1.932713,2.075877,1.073729,1.861131,3.579098,2.648533,7.730852,6.299213,1.503221,2.219041,2.075877,10.952040,2.075877,5.941303,30.923407,10.450966,42.161775,396,33886.0,676,65,57.838225,9.591983,1.288475,24.767359,2016
3,36047008500,14940.45,7281,16.440049,5.219063,2.444719,3.914297,2.197500,2.430985,0.686719,2.540860,0.164813,1.071281,3.007829,1.510781,0.247219,0.000000,0.000000,0.206016,9.284439,3.708282,6.825985,2.087625,0.879000,98.475484,1311,12254.0,3089,25,1.524516,44.897679,4.669688,46.944101,2016


In [ ]:
demo_data_2022_rh = demo_data_2022.copy()
demo_data_2022_rh.head()

,geo_id,median_income,income_less_10000,income_10000_14999,income_15000_19999,income_20000_24999,income_25000_29999,income_30000_34999,income_35000_39999,income_40000_44999,income_45000_49999,income_50000_59999,income_60000_74999,income_75000_99999,income_100000_124999,income_125000_149999,income_150000_199999,income_200000_or_more,high_school_diploma,less_one_year_college,one_year_more_college,bachelors_degree,masters_degree,other_races,commuters_by_public_transportation,income_per_capita,housing_units,vacant_housing_units,median_rent,white_pop,black_pop,asian_pop,hispanic_pop
0,36047005300,88237.500078,3.673657,0.545978,3.314382,-2.655295,2.404052,4.196310,0.495901,-3.414142,-1.693618,-0.285309,1.850636,10.448520,-10.991569,6.437527,4.330317,16.351430,-11.263266,2.990998,7.242160,8.607851,24.315846,43.116579,899.349115,68985.066086,1493.057694,-26.376192,1814.552769,55.304172,22.902085,-8.429385,14.969850
1,36047004700,107139.158564,-2.876878,2.285901,3.749399,1.320397,-0.828882,1.572746,2.191943,0.793929,3.564199,-5.948172,1.608419,-0.551120,11.360169,0.943762,4.455201,14.034931,2.818045,6.819257,8.095929,19.845507,27.456576,37.324998,454.980078,58982.572453,1110.005323,60.361923,2488.071964,65.563607,-4.019905,0.003511,12.200261
2,36047005900,94671.108845,3.538551,-10.871276,1.095397,3.143523,1.999328,5.347873,3.101730,1.425567,5.792755,-7.378379,0.753801,8.125567,14.016596,-0.374333,8.955132,3.470029,10.879021,5.451061,14.567889,48.808714,-0.267597,34.889383,611.299970,47188.241857,902.915155,-14.054450,2677.844003,64.889594,13.321007,-7.625539,18.026466
3,36047008500,9236.186357,24.718412,2.611746,2.599958,3.356741,-0.995334,3.044618,1.654982,4.790598,-2.676764,0.944857,3.582363,2.120697,0.639367,-0.942782,0.247433,0.724769,2.620652,1.210775,1.741608,-3.059114,1.865711,82.803608,1501.530295,17068.899828,3207.473991,-77.730191,332.498273,2.323068,67.007571,12.222600,32.156294


In [ ]:
income_vars = [ 'income_less_10000','income_10000_14999','income_15000_19999','income_20000_24999','income_25000_29999',
                'income_30000_34999','income_35000_39999','income_40000_44999','income_45000_49999','income_50000_59999','income_60000_74999',
                'income_75000_99999','income_100000_124999','income_125000_149999','income_150000_199999','income_200000_or_more']

demo_data_2022_rh['income_dist'] = demo_data_2022_rh[income_vars].apply(calc_dist_from_ideal,div_int=16,axis=1)
demo_data_2017_rh['income_dist'] = demo_data_2017_rh[income_vars].apply(calc_dist_from_ideal,div_int=16,axis=1)
demo_data_2016_rh['income_dist'] = demo_data_2016_rh[income_vars].apply(calc_dist_from_ideal,div_int=16,axis=1)

education_vars = ['high_school_diploma','less_one_year_college','one_year_more_college','bachelors_degree', 'masters_degree']

demo_data_2022_rh['edu_dist'] = demo_data_2022_rh[education_vars].apply(calc_dist_from_ideal,div_int=5,axis=1)
demo_data_2017_rh['edu_dist'] = demo_data_2017_rh[education_vars].apply(calc_dist_from_ideal,div_int=5,axis=1)
demo_data_2016_rh['income_dist'] = demo_data_2016_rh[income_vars].apply(calc_dist_from_ideal,div_int=16,axis=1)

In [ ]:
#Concating Yearly Data to derive time-series variables
demo_data_rh = pd.concat((demo_data_2016_rh,demo_data_2017_rh,demo_data_2022_rh))

In [ ]:
demo_data_rh.tail()

,geo_id,median_income,total_pop,income_less_10000,income_10000_14999,income_15000_19999,income_20000_24999,income_25000_29999,income_30000_34999,income_35000_39999,income_40000_44999,income_45000_49999,income_50000_59999,income_60000_74999,income_75000_99999,income_100000_124999,income_125000_149999,income_150000_199999,income_200000_or_more,high_school_diploma,less_one_year_college,one_year_more_college,bachelors_degree,masters_degree,other_races,commuters_by_public_transportation,income_per_capita,housing_units,vacant_housing_units,white_pop,black_pop,asian_pop,hispanic_pop,year,income_dist,edu_dist,median_rent
3,36047008500,13471.140000,7559.0,15.808969,6.667549,2.447414,2.275433,2.328350,2.434184,1.481677,2.407726,0.185210,0.859902,2.500331,1.084800,0.251356,0.000000,0.277815,0.185210,9.763196,2.844292,6.019315,1.772721,0.701151,98.941659,1236.000000,11901.000000,3140.000000,26.000000,1.058341,47.810557,5.212330,44.304802,2017,0.661046,0.233598,NaN
0,36047005300,88237.500078,NaN,3.673657,0.545978,3.314382,-2.655295,2.404052,4.196310,0.495901,-3.414142,-1.693618,-0.285309,1.850636,10.448520,-10.991569,6.437527,4.330317,16.351430,-11.263266,2.990998,7.242160,8.607851,24.315846,43.116579,899.349115,68985.066086,1493.057694,-26.376192,55.304172,22.902085,-8.429385,14.969850,NaN,0.618687,NaN,1814.552769
1,36047004700,107139.158564,NaN,-2.876878,2.285901,3.749399,1.320397,-0.828882,1.572746,2.191943,0.793929,3.564199,-5.948172,1.608419,-0.551120,11.360169,0.943762,4.455201,14.034931,2.818045,6.819257,8.095929,19.845507,27.456576,37.324998,454.980078,58982.572453,1110.005323,60.361923,65.563607,-4.019905,0.003511,12.200261,NaN,NaN,0.316348,2488.071964
2,36047005900,94671.108845,NaN,3.538551,-10.871276,1.095397,3.143523,1.999328,5.347873,3.101730,1.425567,5.792755,-7.378379,0.753801,8.125567,14.016596,-0.374333,8.955132,3.470029,10.879021,5.451061,14.567889,48.808714,-0.267597,34.889383,611.299970,47188.241857,902.915155,-14.054450,64.889594,13.321007,-7.625539,18.026466,NaN,NaN,0.190132,2677.844003
3,36047008500,9236.186357,NaN,24.718412,2.611746,2.599958,3.356741,-0.995334,3.044618,1.654982,4.790598,-2.676764,0.944857,3.582363,2.120697,0.639367,-0.942782,0.247433,0.724769,2.620652,1.210775,1.741608,-3.059114,1.865711,82.803608,1501.530295,17068.899828,3207.473991,-77.730191,2.323068,67.007571,12.222600,32.156294,NaN,0.710542,0.413438,332.498273


In [ ]:
demo_data_rh.to_csv('redhook_demographic_prediction_10Yr_Percent.csv')

In [ ]:
#Calculate Time Series Variables (difference of population, income, education over years)
def calc_time_series_vars(census_data,variable_list,start_yr,end_yr):
  
  df = demo_data_2022_rh[['geo_id']]
  vars = variable_list
  year = [start_yr,end_yr]

  for j in vars:
      df['{}_{}_{}'.format(end_yr,start_yr,j)] = (census_data.loc[census_data['year'] == str(end_yr), j] - census_data.loc[census_data['year'] == str(start_yr), j])

  return df

In [ ]:
var_list = ['median_income','median_rent','income_dist','edu_dist','other_races','commuters_by_public_transportation','income_per_capita','housing_units','vacant_housing_units','bachelors_degree']
df = calc_time_series_vars(demo_data_rh,var_list,2017,2022)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
df.head()

,geo_id,2022_2017_median_income,2022_2017_median_rent,2022_2017_income_dist,2022_2017_edu_dist,2022_2017_other_races,2022_2017_commuters_by_public_transportation,2022_2017_income_per_capita,2022_2017_housing_units,2022_2017_vacant_housing_units,2022_2017_bachelors_degree
0,36047005300,-241.530822,NaN,0.198901,0.287012,94.413245,199.051668,21926.838440,173.314816,-109.681906,138.060110
1,36047004700,-11522.486485,NaN,NaN,-0.067625,170.264156,-173.384460,-9299.253349,214.356759,-11.264447,-11.195519
2,36047005900,12033.214941,NaN,NaN,-0.027196,187.867740,78.189595,822.062197,158.254830,-51.556596,151.221743
3,36047008500,-1710.671304,NaN,0.027318,0.108046,-897.245871,166.776227,3239.579385,12.914013,-55.333173,-189.172105


In [ ]:
#Calculating time changes between 2011-2012
df2 = calc_time_series_vars(demo_data_rh,var_list,2016,2017)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
df2.head()

,geo_id,2017_2016_median_income,2017_2016_median_rent,2017_2016_income_dist,2017_2016_edu_dist,2017_2016_other_races,2017_2016_commuters_by_public_transportation,2017_2016_income_per_capita,2017_2016_housing_units,2017_2016_vacant_housing_units,2017_2016_bachelors_degree
0,36047005300,10968.75,NaN,0.031706,NaN,-10.0,-19.0,927.0,-12.0,16.0,11.0
1,36047004700,-4845.54,NaN,0.091227,NaN,-88.0,-7.0,2819.0,-17.0,30.0,-34.0
2,36047005900,2605.26,NaN,0.109861,NaN,-61.0,49.0,6500.0,-4.0,1.0,14.0
3,36047008500,-1469.31,NaN,0.014170,NaN,309.0,-75.0,-353.0,51.0,1.0,-18.0


In [ ]:
#Merging study datasets
df = df.merge(df2,how='left',on='geo_id')
df.head()

,geo_id,2022_2017_median_income,2022_2017_median_rent,2022_2017_income_dist,2022_2017_edu_dist,2022_2017_other_races,2022_2017_commuters_by_public_transportation,2022_2017_income_per_capita,2022_2017_housing_units,2022_2017_vacant_housing_units,2022_2017_bachelors_degree,2017_2016_median_income,2017_2016_median_rent,2017_2016_income_dist,2017_2016_edu_dist,2017_2016_other_races,2017_2016_commuters_by_public_transportation,2017_2016_income_per_capita,2017_2016_housing_units,2017_2016_vacant_housing_units,2017_2016_bachelors_degree
0,36047005300,-241.530822,NaN,0.198901,0.287012,94.413245,199.051668,21926.838440,173.314816,-109.681906,138.060110,10968.75,NaN,0.031706,NaN,-10.0,-19.0,927.0,-12.0,16.0,11.0
1,36047004700,-11522.486485,NaN,NaN,-0.067625,170.264156,-173.384460,-9299.253349,214.356759,-11.264447,-11.195519,-4845.54,NaN,0.091227,NaN,-88.0,-7.0,2819.0,-17.0,30.0,-34.0
2,36047005900,12033.214941,NaN,NaN,-0.027196,187.867740,78.189595,822.062197,158.254830,-51.556596,151.221743,2605.26,NaN,0.109861,NaN,-61.0,49.0,6500.0,-4.0,1.0,14.0
3,36047008500,-1710.671304,NaN,0.027318,0.108046,-897.245871,166.776227,3239.579385,12.914013,-55.333173,-189.172105,-1469.31,NaN,0.014170,NaN,309.0,-75.0,-353.0,51.0,1.0,-18.0


In [ ]:
#Concating SVI Variables
svi_data = pd.concat((svi_2014,svi_2016,svi_2018))
svi_data.head()

,FID,AFFGEOID,TRACTCE,ST,STATE,ST_ABBR,STCNTY,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,M_HU,E_HH,M_HH,E_POV,M_POV,E_UNEMP,M_UNEMP,E_PCI,M_PCI,E_NOHSDP,M_NOHSDP,E_AGE65,M_AGE65,E_AGE17,M_AGE17,E_DISABL,M_DISABL,E_SNGPNT,M_SNGPNT,E_MINRTY,M_MINRTY,E_LIMENG,M_LIMENG,E_MUNIT,M_MUNIT,E_MOBILE,...,RPL_THEME2,EPL_MINRTY,EPL_LIMENG,SPL_THEME3,RPL_THEME3,EPL_MUNIT,EPL_MOBILE,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,F_POV,F_UNEMP,F_PCI,F_NOHSDP,F_THEME1,F_AGE65,F_AGE17,F_DISABL,F_SNGPNT,F_THEME2,F_MINRTY,F_LIMENG,F_THEME3,F_MUNIT,F_MOBILE,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP,year
0,-1.0,1400000US36001000100,100.0,36,New York,NY,36001,Albany,36001000100,"Census Tract 1, Albany County, New York",0.914080,2201.0,315.0,975.0,51.0,814.0,85.0,877.0,307.0,151.0,79.0,18612.0,3209.0,299.0,110.0,304.0,84.0,648.0,192.0,286.0,90.0,205.0,87.8,1782.0,349.3,9.0,44.5,35.0,32.7,0.0,...,0.9690,0.7572,0.1986,0.9558,0.4640,0.3929,0.0000,0.7221,0.5804,0.7099,2.4053,0.5731,9.8134,0.8573,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,237.0,98.0,10.8,4.6,3625.0,2014
1,-1.0,1400000US36001000200,200.0,36,New York,NY,36001,Albany,36001000200,"Census Tract 2, Albany County, New York",0.797363,5198.0,764.0,2768.0,195.0,2087.0,245.0,2206.0,665.0,228.0,137.0,16624.0,3069.0,501.0,198.0,389.0,119.0,1536.0,551.0,383.0,169.0,395.0,155.3,4753.0,782.3,41.0,75.7,543.0,171.6,0.0,...,0.5558,0.8290,0.2909,1.1199,0.5456,0.6699,0.0000,0.3853,0.7154,0.7568,2.5274,0.6340,8.9258,0.7651,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,644.0,407.0,12.4,7.2,5027.0,2014
2,-1.0,1400000US36001000300,300.0,36,New York,NY,36001,Albany,36001000300,"Census Tract 3, Albany County, New York",2.231410,5519.0,785.0,2673.0,148.0,2392.0,161.0,1906.0,483.0,218.0,133.0,22048.0,2841.0,483.0,166.0,927.0,205.0,1434.0,435.0,535.0,233.0,351.0,124.8,3115.0,850.4,13.0,62.6,265.0,155.7,0.0,...,0.8938,0.6575,0.1429,0.8004,0.3898,0.5515,0.0000,0.3594,0.6239,0.8414,2.3761,0.5586,8.6871,0.7354,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,720.0,633.0,13.6,10.8,10514.0,2014
3,-1.0,1400000US36001000401,401.0,36,New York,NY,36001,Albany,36001000401,"Census Tract 4.01, Albany County, New York",3.481864,2367.0,212.0,1125.0,51.0,998.0,84.0,72.0,36.0,74.0,50.0,38497.0,5302.0,148.0,75.0,1182.0,184.0,184.0,48.0,423.0,110.0,28.0,28.2,314.0,323.2,27.0,47.2,272.0,63.0,21.0,...,0.5941,0.2871,0.3476,0.6347,0.3081,0.7085,0.8261,0.0000,0.4502,0.9721,2.9568,0.8114,6.9030,0.4709,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0,107.0,70.0,5.9,3.8,12048.0,2014
4,-1.0,1400000US36001000403,403.0,36,New York,NY,36001,Albany,36001000403,"Census Tract 4.03, Albany County, New York",1.211858,4578.0,458.0,2538.0,99.0,2290.0,155.0,1018.0,422.0,187.0,122.0,33468.0,4051.0,299.0,155.0,698.0,160.0,408.0,193.0,671.0,248.0,101.0,84.1,1495.0,610.7,98.0,114.5,73.0,62.4,0.0,...,0.2838,0.5107,0.4728,0.9835,0.4793,0.3633,0.0000,0.2102,0.5617,0.4589,1.5941,0.2328,6.1608,0.3408,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,515.0,212.0,11.2,4.9,5194.0,2014


In [ ]:
svi_data_rh = svi_data.copy()
svi_data_rh = svi_data[['FIPS','EP_POV','EP_MUNIT','EP_MOBILE','EP_SNGPNT','EP_CROWD','EP_MINRTY','year']]
svi_data_rh.year.value_counts()

2014    4906
2018    4906
2016    4906
Name: year, dtype: int64

In [ ]:
svi_data_rh = svi_data_rh.rename(columns={'FIPS':'geo_id'})
svi_data_rh = pd.merge(redhook_df,svi_data_rh,on='geo_id',how='left')
svi_data_rh.head()

,geo_id,EP_POV,EP_MUNIT,EP_MOBILE,EP_SNGPNT,EP_CROWD,EP_MINRTY,year
0,36047005300,20.5,21.6,0.0,6.3,11.9,55.0,2014
1,36047005300,19.6,16.2,0.0,11.2,10.4,50.4,2016
2,36047005300,16.1,17.5,0.0,12.2,7.5,45.0,2018
3,36047004700,10.2,60.0,0.0,10.8,6.0,51.7,2014
4,36047004700,7.2,61.4,0.0,5.7,1.5,45.3,2016


In [ ]:
#Median Income

svi_data_rh_test = svi_data_rh.copy()
svi_data_rh_test = svi_data_rh_test.rename(columns={'year': 'ds', 'EP_POV':'y'})

geoid = list(redhook_df['geo_id'])
EP_POV_2027 = []

from fbprophet import Prophet
grouped = svi_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=8,freq='Y')
    forecast = m.predict(future)
    EP_POV_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 1.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 1.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 1.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
#Median Income

svi_data_rh_test = svi_data_rh.copy()
svi_data_rh_test = svi_data_rh_test.rename(columns={'year': 'ds', 'EP_MUNIT':'y'})

geoid = list(redhook_df['geo_id'])
EP_MUNIT_2027 = []

from fbprophet import Prophet
grouped = svi_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=8,freq='Y')
    forecast = m.predict(future)
    EP_MUNIT_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 1.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 1.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 1.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
#Median Income

svi_data_rh_test = svi_data_rh.copy()
svi_data_rh_test = svi_data_rh_test.rename(columns={'year': 'ds', 'EP_MOBILE':'y'})

geoid = list(redhook_df['geo_id'])
EP_MOBILE_2027 = []

from fbprophet import Prophet
grouped = svi_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=8,freq='Y')
    forecast = m.predict(future)
    EP_MOBILE_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 1.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 1.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 1.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
#Median Income

svi_data_rh_test = svi_data_rh.copy()
svi_data_rh_test = svi_data_rh_test.rename(columns={'year': 'ds', 'EP_MOBILE':'y'})

geoid = list(redhook_df['geo_id'])
EP_MOBILE_2027 = []

from fbprophet import Prophet
grouped = svi_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=8,freq='Y')
    forecast = m.predict(future)
    EP_MOBILE_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 1.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 1.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 1.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
#Median Income

svi_data_rh_test = svi_data_rh.copy()
svi_data_rh_test = svi_data_rh_test.rename(columns={'year': 'ds', 'EP_SNGPNT':'y'})

geoid = list(redhook_df['geo_id'])
EP_SNGPNT_2027 = []

from fbprophet import Prophet
grouped = svi_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=8,freq='Y')
    forecast = m.predict(future)
    EP_SNGPNT_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 1.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 1.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 1.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
#Median Income

svi_data_rh_test = svi_data_rh.copy()
svi_data_rh_test = svi_data_rh_test.rename(columns={'year': 'ds', 'EP_CROWD':'y'})

geoid = list(redhook_df['geo_id'])
EP_CROWD_2027 = []

from fbprophet import Prophet
grouped = svi_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=8,freq='Y')
    forecast = m.predict(future)
    EP_CROWD_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 1.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 1.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 1.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
#Median Income

svi_data_rh_test = svi_data_rh.copy()
svi_data_rh_test = svi_data_rh_test.rename(columns={'year': 'ds', 'EP_MINRTY':'y'})

geoid = list(redhook_df['geo_id'])
EP_MINRTY_2027 = []

from fbprophet import Prophet
grouped = svi_data_rh_test.groupby('geo_id')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet(yearly_seasonality = True)
    m.fit(group)
    future = m.make_future_dataframe(periods=8,freq='Y')
    forecast = m.predict(future)
    EP_MINRTY_2027.append(float(forecast.yhat.tail(1)))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 1.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 1.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 1.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [ ]:
svi_data_2027 = pd.DataFrame({'geo_id':geoid,'EP_POV':EP_POV_2027,'EP_MUNIT':EP_MUNIT_2027,'EP_MOBILE':EP_MOBILE_2027,'EP_SNGPNT':EP_SNGPNT_2027,'EP_CROWD':EP_CROWD_2027,'EP_MINRTY':EP_MINRTY_2027})

svi_data_2027['year'] = '2027'
svi_data_2027

,geo_id,EP_POV,EP_MUNIT,EP_MOBILE,EP_SNGPNT,EP_CROWD,EP_MINRTY,year
0,36047005300,-0.736593,64.358844,0.0,-11.270419,-12.191306,24.655004,2027
1,36047004700,9.745394,2.468118,0.0,27.845194,0.217938,25.188624,2027
2,36047005900,6.784185,32.120378,0.0,8.650038,3.849771,60.559667,2027
3,36047008500,31.389705,100.580671,0.0,37.501394,34.635230,98.742492,2027


In [ ]:
svi_data_rh.to_csv('redhook_svi_predictions_10year_percent.csv')

In [ ]:
#Calculate Time Series Variables (difference of population, income, education over years)
def calc_time_series_vars_svi(census_data,variable_list,start_yr,end_yr):
  
  df = svi_data_2022[['geo_id']]
  vars = variable_list
  year = [start_yr,end_yr]

  for j in vars:
      df['{}_{}_{}'.format(end_yr,start_yr,j)] = (census_data.loc[census_data['year'] == str(end_yr), j] - census_data.loc[census_data['year'] == str(start_yr), j])

  return df

In [ ]:
svi_data_rh = pd.concat((svi_data_rh,svi_data_2022))

In [ ]:
svi_var_list = ['EP_POV','EP_MUNIT','EP_MOBILE']
df3 = calc_time_series_vars_svi(svi_data_rh,svi_var_list,2018,2022)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
df3.head()

,geo_id,2022_2018_EP_POV,2022_2018_EP_MUNIT,2022_2018_EP_MOBILE
0,36047005300,NaN,NaN,NaN
1,36047004700,NaN,NaN,NaN
2,36047005900,0.284185,14.920378,0.0
3,36047008500,NaN,NaN,NaN


In [ ]:
df = df.merge(df3,how='left',on='geo_id')

In [ ]:
df.head()

,geo_id,2022_2017_median_income,2022_2017_median_rent,2022_2017_income_dist,2022_2017_edu_dist,2022_2017_other_races,2022_2017_commuters_by_public_transportation,2022_2017_income_per_capita,2022_2017_housing_units,2022_2017_vacant_housing_units,2022_2017_bachelors_degree,2017_2016_median_income,2017_2016_median_rent,2017_2016_income_dist,2017_2016_edu_dist,2017_2016_other_races,2017_2016_commuters_by_public_transportation,2017_2016_income_per_capita,2017_2016_housing_units,2017_2016_vacant_housing_units,2017_2016_bachelors_degree,2022_2018_EP_POV,2022_2018_EP_MUNIT,2022_2018_EP_MOBILE
0,36047005300,-241.530822,NaN,0.198901,0.287012,94.413245,199.051668,21926.838440,173.314816,-109.681906,138.060110,10968.75,NaN,0.031706,NaN,-10.0,-19.0,927.0,-12.0,16.0,11.0,NaN,NaN,NaN
1,36047004700,-11522.486485,NaN,NaN,-0.067625,170.264156,-173.384460,-9299.253349,214.356759,-11.264447,-11.195519,-4845.54,NaN,0.091227,NaN,-88.0,-7.0,2819.0,-17.0,30.0,-34.0,NaN,NaN,NaN
2,36047005900,12033.214941,NaN,NaN,-0.027196,187.867740,78.189595,822.062197,158.254830,-51.556596,151.221743,2605.26,NaN,0.109861,NaN,-61.0,49.0,6500.0,-4.0,1.0,14.0,0.284185,14.920378,0.0
3,36047008500,-1710.671304,NaN,0.027318,0.108046,-897.245871,166.776227,3239.579385,12.914013,-55.333173,-189.172105,-1469.31,NaN,0.014170,NaN,309.0,-75.0,-353.0,51.0,1.0,-18.0,NaN,NaN,NaN


In [ ]:
NYC_COUNTIES = [
    "005", # Bronx
    "047", # Kings (Brooklyn)
    "061", # New York County (Manhattan)
    "081", # Queens
    "085", # Richmond (Staten Island)
]

In [ ]:
spatial_data = gpd.read_file('tl_2019_36_tract.shp')
spatial_data['GEOID'] = spatial_data['GEOID'].astype(int)
spatial_data= spatial_data[spatial_data['COUNTYFP'].isin(NYC_COUNTIES)]
spatial_data= spatial_data[spatial_data['GEOID'].isin(geoid)]
spatial_data.head()

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
1013,36,047,005300,36047005300,53,Census Tract 53,G5020,S,1651843,5374846,+40.6708307,-074.0291182,"POLYGON ((-74.04894 40.66270, -74.04822 40.663..."
4658,36,047,005900,36047005900,59,Census Tract 59,G5020,S,355163,0,+40.6794028,-074.0061193,"POLYGON ((-74.01170 40.67936, -74.01045 40.680..."
4660,36,047,004700,36047004700,47,Census Tract 47,G5020,S,283960,0,+40.6883662,-074.0018662,"POLYGON ((-74.00843 40.68602, -74.00816 40.686..."
4689,36,047,008500,36047008500,85,Census Tract 85,G5020,S,243576,0,+40.6758570,-074.0064716,"POLYGON ((-74.01079 40.67683, -74.01017 40.677..."


In [ ]:
#Calculating queen weights for each GeoID (census tract)
w = Queen.from_dataframe(spatial_data,idVariable='GEOID')
w.transform = 'R'

In [ ]:
spatial_data = spatial_data.merge(df, how='inner',left_on='GEOID',right_on='geo_id')
spatial_data.head()

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,geo_id,2022_2017_median_income,2022_2017_median_rent,2022_2017_income_dist,2022_2017_edu_dist,2022_2017_other_races,2022_2017_commuters_by_public_transportation,2022_2017_income_per_capita,2022_2017_housing_units,2022_2017_vacant_housing_units,2022_2017_bachelors_degree,2017_2016_median_income,2017_2016_median_rent,2017_2016_income_dist,2017_2016_edu_dist,2017_2016_other_races,2017_2016_commuters_by_public_transportation,2017_2016_income_per_capita,2017_2016_housing_units,2017_2016_vacant_housing_units,2017_2016_bachelors_degree,2022_2018_EP_POV,2022_2018_EP_MUNIT,2022_2018_EP_MOBILE
0,36,047,005300,36047005300,53,Census Tract 53,G5020,S,1651843,5374846,+40.6708307,-074.0291182,"POLYGON ((-74.04894 40.66270, -74.04822 40.663...",36047005300,-241.530822,NaN,0.198901,0.287012,94.413245,199.051668,21926.838440,173.314816,-109.681906,138.060110,10968.75,NaN,0.031706,NaN,-10.0,-19.0,927.0,-12.0,16.0,11.0,NaN,NaN,NaN
1,36,047,005900,36047005900,59,Census Tract 59,G5020,S,355163,0,+40.6794028,-074.0061193,"POLYGON ((-74.01170 40.67936, -74.01045 40.680...",36047005900,12033.214941,NaN,NaN,-0.027196,187.867740,78.189595,822.062197,158.254830,-51.556596,151.221743,2605.26,NaN,0.109861,NaN,-61.0,49.0,6500.0,-4.0,1.0,14.0,0.284185,14.920378,0.0
2,36,047,004700,36047004700,47,Census Tract 47,G5020,S,283960,0,+40.6883662,-074.0018662,"POLYGON ((-74.00843 40.68602, -74.00816 40.686...",36047004700,-11522.486485,NaN,NaN,-0.067625,170.264156,-173.384460,-9299.253349,214.356759,-11.264447,-11.195519,-4845.54,NaN,0.091227,NaN,-88.0,-7.0,2819.0,-17.0,30.0,-34.0,NaN,NaN,NaN
3,36,047,008500,36047008500,85,Census Tract 85,G5020,S,243576,0,+40.6758570,-074.0064716,"POLYGON ((-74.01079 40.67683, -74.01017 40.677...",36047008500,-1710.671304,NaN,0.027318,0.108046,-897.245871,166.776227,3239.579385,12.914013,-55.333173,-189.172105,-1469.31,NaN,0.014170,NaN,309.0,-75.0,-353.0,51.0,1.0,-18.0,NaN,NaN,NaN


In [ ]:
#Checking for null values
spatial_data.isnull().sum(axis = 0)

STATEFP                                         0
COUNTYFP                                        0
TRACTCE                                         0
GEOID                                           0
NAME                                            0
NAMELSAD                                        0
MTFCC                                           0
FUNCSTAT                                        0
ALAND                                           0
AWATER                                          0
INTPTLAT                                        0
INTPTLON                                        0
geometry                                        0
geo_id                                          0
2022_2017_median_income                         0
2022_2017_median_rent                           4
2022_2017_income_dist                           2
2022_2017_edu_dist                              0
2022_2017_other_races                           0
2022_2017_commuters_by_public_transportation    0


In [ ]:
#Fill NAs with 0 for spatial lag calculations
spatial_data = spatial_data.fillna(0)

In [ ]:
spatial_data.shape

(4, 37)

In [ ]:
spatial_data

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,geo_id,2022_2017_median_income,2022_2017_median_rent,2022_2017_income_dist,2022_2017_edu_dist,2022_2017_other_races,2022_2017_commuters_by_public_transportation,2022_2017_income_per_capita,2022_2017_housing_units,2022_2017_vacant_housing_units,2022_2017_bachelors_degree,2017_2016_median_income,2017_2016_median_rent,2017_2016_income_dist,2017_2016_edu_dist,2017_2016_other_races,2017_2016_commuters_by_public_transportation,2017_2016_income_per_capita,2017_2016_housing_units,2017_2016_vacant_housing_units,2017_2016_bachelors_degree,2022_2018_EP_POV,2022_2018_EP_MUNIT,2022_2018_EP_MOBILE
0,36,047,005300,36047005300,53,Census Tract 53,G5020,S,1651843,5374846,+40.6708307,-074.0291182,"POLYGON ((-74.04894 40.66270, -74.04822 40.663...",36047005300,-241.530822,0.0,0.198901,0.287012,94.413245,199.051668,21926.838440,173.314816,-109.681906,138.060110,10968.75,0.0,0.031706,0.0,-10.0,-19.0,927.0,-12.0,16.0,11.0,0.000000,0.000000,0.0
1,36,047,005900,36047005900,59,Census Tract 59,G5020,S,355163,0,+40.6794028,-074.0061193,"POLYGON ((-74.01170 40.67936, -74.01045 40.680...",36047005900,12033.214941,0.0,0.000000,-0.027196,187.867740,78.189595,822.062197,158.254830,-51.556596,151.221743,2605.26,0.0,0.109861,0.0,-61.0,49.0,6500.0,-4.0,1.0,14.0,0.284185,14.920378,0.0
2,36,047,004700,36047004700,47,Census Tract 47,G5020,S,283960,0,+40.6883662,-074.0018662,"POLYGON ((-74.00843 40.68602, -74.00816 40.686...",36047004700,-11522.486485,0.0,0.000000,-0.067625,170.264156,-173.384460,-9299.253349,214.356759,-11.264447,-11.195519,-4845.54,0.0,0.091227,0.0,-88.0,-7.0,2819.0,-17.0,30.0,-34.0,0.000000,0.000000,0.0
3,36,047,008500,36047008500,85,Census Tract 85,G5020,S,243576,0,+40.6758570,-074.0064716,"POLYGON ((-74.01079 40.67683, -74.01017 40.677...",36047008500,-1710.671304,0.0,0.027318,0.108046,-897.245871,166.776227,3239.579385,12.914013,-55.333173,-189.172105,-1469.31,0.0,0.014170,0.0,309.0,-75.0,-353.0,51.0,1.0,-18.0,0.000000,0.000000,0.0


In [ ]:
#Calculating spatial weights
import libpysal as lps

spatial_data['lagged_spatial_income'] = lps.weights.lag_spatial(w, spatial_data['2017_2016_median_income'])
spatial_data['lagged_income_dist'] = lps.weights.lag_spatial(w, spatial_data['2017_2016_income_dist'])
spatial_data['lagged_other_races'] = lps.weights.lag_spatial(w, spatial_data['2017_2016_other_races'])
#spatial_data['lagged_edu_dist'] = lps.weights.lag_spatial(w, spatial_data['2012_2011_edu_dist'])
spatial_data['lagged_public_commute'] = lps.weights.lag_spatial(w, spatial_data['2017_2016_commuters_by_public_transportation'])
spatial_data['lagged_income_per_capita'] = lps.weights.lag_spatial(w, spatial_data['2017_2016_income_per_capita'])
spatial_data['lagged_housing_units'] = lps.weights.lag_spatial(w, spatial_data['2017_2016_housing_units'])
spatial_data['lagged_vacant_housing_units'] = lps.weights.lag_spatial(w, spatial_data['2017_2016_vacant_housing_units'])
spatial_data['lagged_spatial_rent'] = lps.weights.lag_spatial(w, spatial_data['2017_2016_median_rent'])

In [ ]:
spatial_data_rh = spatial_data.copy()
spatial_data_rh.head()

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,geo_id,2022_2017_median_income,2022_2017_median_rent,2022_2017_income_dist,2022_2017_edu_dist,2022_2017_other_races,2022_2017_commuters_by_public_transportation,2022_2017_income_per_capita,2022_2017_housing_units,2022_2017_vacant_housing_units,2022_2017_bachelors_degree,2017_2016_median_income,2017_2016_median_rent,2017_2016_income_dist,2017_2016_edu_dist,2017_2016_other_races,2017_2016_commuters_by_public_transportation,2017_2016_income_per_capita,2017_2016_housing_units,2017_2016_vacant_housing_units,2017_2016_bachelors_degree,2022_2018_EP_POV,2022_2018_EP_MUNIT,2022_2018_EP_MOBILE,lagged_spatial_income,lagged_income_dist,lagged_other_races,lagged_public_commute,lagged_income_per_capita,lagged_housing_units,lagged_vacant_housing_units,lagged_spatial_rent
0,36,047,005300,36047005300,53,Census Tract 53,G5020,S,1651843,5374846,+40.6708307,-074.0291182,"POLYGON ((-74.04894 40.66270, -74.04822 40.663...",36047005300,-241.530822,0.0,0.198901,0.287012,94.413245,199.051668,21926.838440,173.314816,-109.681906,138.060110,10968.75,0.0,0.031706,0.0,-10.0,-19.0,927.0,-12.0,16.0,11.0,0.000000,0.000000,0.0,2605.260,0.109861,-61.000000,49.000000,6500.0,-4.000000,1.000000,0.0
1,36,047,005900,36047005900,59,Census Tract 59,G5020,S,355163,0,+40.6794028,-074.0061193,"POLYGON ((-74.01170 40.67936, -74.01045 40.680...",36047005900,12033.214941,0.0,0.000000,-0.027196,187.867740,78.189595,822.062197,158.254830,-51.556596,151.221743,2605.26,0.0,0.109861,0.0,-61.0,49.0,6500.0,-4.0,1.0,14.0,0.284185,14.920378,0.0,1551.300,0.045701,70.333333,-33.666667,1131.0,7.333333,15.666667,0.0
2,36,047,004700,36047004700,47,Census Tract 47,G5020,S,283960,0,+40.6883662,-074.0018662,"POLYGON ((-74.00843 40.68602, -74.00816 40.686...",36047004700,-11522.486485,0.0,0.000000,-0.067625,170.264156,-173.384460,-9299.253349,214.356759,-11.264447,-11.195519,-4845.54,0.0,0.091227,0.0,-88.0,-7.0,2819.0,-17.0,30.0,-34.0,0.000000,0.000000,0.0,567.975,0.062015,124.000000,-13.000000,3073.5,23.500000,1.000000,0.0
3,36,047,008500,36047008500,85,Census Tract 85,G5020,S,243576,0,+40.6758570,-074.0064716,"POLYGON ((-74.01079 40.67683, -74.01017 40.677...",36047008500,-1710.671304,0.0,0.027318,0.108046,-897.245871,166.776227,3239.579385,12.914013,-55.333173,-189.172105,-1469.31,0.0,0.014170,0.0,309.0,-75.0,-353.0,51.0,1.0,-18.0,0.000000,0.000000,0.0,-1120.140,0.100544,-74.500000,21.000000,4659.5,-10.500000,15.500000,0.0


# Getting Model Parameters

In [ ]:
#Loading demographic data 
demo_data_2018 = pd.read_csv('acs_ct_2018.csv')
demo_data_2018['year'] = '2018'
demo_data_2017 = pd.read_csv('acs_ct_2017.csv')
demo_data_2017['year'] = '2017'
demo_data_2016 = pd.read_csv('acs_ct_2016.csv')
demo_data_2016['year'] = '2016'
demo_data_2015 = pd.read_csv('acs_ct_2015.csv')
demo_data_2015['year'] = '2015'
demo_data_2014 = pd.read_csv('acs_ct_2014.csv')
demo_data_2014['year'] = '2014'
demo_data_2013 = pd.read_csv('acs_ct_2013.csv')
demo_data_2013['year'] = '2013'
demo_data_2012 = pd.read_csv('acs_ct_2012.csv')
demo_data_2012['year'] = '2012'
demo_data_2011 = pd.read_csv('acs_ct_2011.csv')
demo_data_2011['year'] = '2011'
demo_data_2010 = pd.read_csv('acs_ct_2010.csv')
demo_data_2010['year'] = '2010'

In [ ]:
#Loading social vulnerability data
svi_2018 = pd.read_csv('NewYork_2018.csv')
svi_2018['year'] = '2018'
svi_2016 = pd.read_csv('NewYork_2016.csv')
svi_2016['year'] = '2016'
svi_2014 = pd.read_csv('NewYork_2014.csv')
svi_2014['year'] = '2014'
svi_2010 = pd.read_csv('NewYork_2010.csv')
svi_2016['year'] = '2010'
svi_2000 = pd.read_csv('NewYork_2000.csv')
svi_2000['year'] = '2000'

In [ ]:
#Applying hellinger distance for all income variables

income_vars = [ 'income_less_10000','income_10000_14999','income_15000_19999','income_20000_24999','income_25000_29999',
                'income_30000_34999','income_35000_39999','income_40000_44999','income_45000_49999','income_50000_59999','income_60000_74999',
                'income_75000_99999','income_100000_124999','income_125000_149999','income_150000_199999','income_200000_or_more']

demo_data_2018['income_dist'] = demo_data_2018[income_vars].apply(calc_dist_from_ideal,div_int=16,axis=1)
demo_data_2017['income_dist'] = demo_data_2017[income_vars].apply(calc_dist_from_ideal,div_int=16,axis=1)
demo_data_2016['income_dist'] = demo_data_2016[income_vars].apply(calc_dist_from_ideal,div_int=16,axis=1)
demo_data_2015['income_dist'] = demo_data_2015[income_vars].apply(calc_dist_from_ideal,div_int=16,axis=1)
demo_data_2014['income_dist'] = demo_data_2014[income_vars].apply(calc_dist_from_ideal,div_int=16,axis=1)
demo_data_2013['income_dist'] = demo_data_2013[income_vars].apply(calc_dist_from_ideal,div_int=16,axis=1)
demo_data_2012['income_dist'] = demo_data_2012[income_vars].apply(calc_dist_from_ideal,div_int=16,axis=1)
demo_data_2011['income_dist'] = demo_data_2011[income_vars].apply(calc_dist_from_ideal,div_int=16,axis=1)
demo_data_2010['income_dist'] = demo_data_2010[income_vars].apply(calc_dist_from_ideal,div_int=16,axis=1)

In [ ]:
demo_data_2017.head()

,geo_id,nonfamily_households,family_households,median_year_structure_built,rent_burden_not_computed,rent_over_50_percent,rent_40_to_50_percent,rent_35_to_40_percent,rent_30_to_35_percent,rent_25_to_30_percent,rent_20_to_25_percent,rent_15_to_20_percent,rent_10_to_15_percent,rent_under_10_percent,total_pop,male_pop,female_pop,median_age,white_pop,black_pop,asian_pop,hispanic_pop,amerindian_pop,other_race_pop,two_or_more_races_pop,not_hispanic_pop,commuters_by_public_transportation,households,median_income,income_per_capita,housing_units,vacant_housing_units,vacant_housing_units_for_rent,vacant_housing_units_for_sale,median_rent,percent_income_spent_on_rent,owner_occupied_housing_units,million_dollar_housing_units,mortgaged_housing_units,families_with_young_children,...,occupation_sales_office,occupation_services,one_car,two_cars,three_cars,pop_25_64,pop_determined_poverty_status,population_1_year_and_over,population_3_years_over,poverty,sales_office_employed,some_college_and_associates_degree,walked_to_work,worked_at_home,workers_16_and_over,associates_degree,bachelors_degree,high_school_diploma,less_one_year_college,masters_degree,one_year_more_college,pop_25_years_over,commute_35_44_mins,commute_60_more_mins,commute_less_10_mins,commuters_16_over,hispanic_any_race,pop_5_years_over,speak_only_english_at_home,speak_spanish_at_home,speak_spanish_at_home_low_english,pop_15_and_over,pop_never_married,pop_now_married,pop_separated,pop_widowed,pop_divorced,do_date,year,income_dist
0,72031050111,277,308,1974.0,13,53,8,13,38,31,23,11,7,0,1168,629,539,52.6,70,0,0,1098,0,0,0,70,0,585,50368.0,32528.0,1236,651,40,10,979.0,32.6,388,0,190,36,...,114,78,269,218,72,633,1168,NaN,1147,124,114,NaN,19,53,542,58,323,87,44,95,158,960,6,37,20,489,1098,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20132017,2017,0.070300
1,72127010022,309,824,1974.0,64,43,12,20,0,0,0,39,47,0,2865,1252,1613,52.8,39,0,0,2826,0,0,0,39,36,1133,52946.0,29942.0,1507,374,28,75,615.0,19.3,908,0,414,113,...,495,31,308,415,188,1462,2860,NaN,2829,200,495,NaN,0,99,1272,186,808,218,72,365,185,2246,50,134,55,1173,2826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20132017,2017,0.120734
2,72111730700,309,912,1985.0,235,38,0,0,0,0,0,0,0,15,4103,2041,2062,32.9,0,0,0,4103,0,0,0,0,0,1221,16688.0,8266.0,1522,301,14,24,286.0,50.0,933,0,337,347,...,194,259,479,347,162,2160,4103,NaN,3886,2593,194,NaN,8,0,1010,341,441,802,29,0,194,2592,77,23,119,1010,4103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20132017,2017,0.582836
3,72113072201,345,994,1972.0,364,57,0,0,10,116,33,16,0,18,3846,1736,2110,43.2,0,0,0,3846,0,0,0,0,0,1339,14127.0,10500.0,1498,159,9,0,99.0,27.5,725,0,163,309,...,222,257,542,324,102,1722,3846,NaN,3681,2025,222,NaN,9,0,849,200,458,576,30,86,453,2601,82,60,81,849,3846,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20132017,2017,0.641039
4,72113073005,231,695,1975.0,46,20,0,0,0,0,0,19,19,0,2822,1255,1567,38.6,0,0,0,2822,0,0,0,0,0,926,28382.0,12435.0,1058,132,18,44,476.0,17.6,822,0,374,102,...,245,120,434,314,138,1378,2822,NaN,2790,666,245,NaN,0,13,818,224,568,583,24,132,195,2038,20,98,0,805,2822,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20132017,2017,0.336173


In [ ]:
#Applying hellinger distance for all education variables

education_vars = ['high_school_diploma','less_one_year_college','one_year_more_college','bachelors_degree', 'masters_degree']

demo_data_2018['edu_dist'] = demo_data_2018[education_vars].apply(calc_dist_from_ideal,div_int=5,axis=1)
demo_data_2017['edu_dist'] = demo_data_2017[education_vars].apply(calc_dist_from_ideal,div_int=5,axis=1)
demo_data_2016['edu_dist'] = demo_data_2016[education_vars].apply(calc_dist_from_ideal,div_int=5,axis=1)
demo_data_2015['edu_dist'] = demo_data_2015[education_vars].apply(calc_dist_from_ideal,div_int=5,axis=1)
demo_data_2014['edu_dist'] = demo_data_2014[education_vars].apply(calc_dist_from_ideal,div_int=5,axis=1)
demo_data_2013['edu_dist'] = demo_data_2013[education_vars].apply(calc_dist_from_ideal,div_int=5,axis=1)
demo_data_2012['edu_dist'] = demo_data_2012[education_vars].apply(calc_dist_from_ideal,div_int=5,axis=1)
demo_data_2011['edu_dist'] = demo_data_2011[education_vars].apply(calc_dist_from_ideal,div_int=5,axis=1)
demo_data_2010['edu_dist'] = demo_data_2010[education_vars].apply(calc_dist_from_ideal,div_int=5,axis=1)

In [ ]:
demo_data_2016.head()

,geo_id,nonfamily_households,family_households,median_year_structure_built,rent_burden_not_computed,rent_over_50_percent,rent_40_to_50_percent,rent_35_to_40_percent,rent_30_to_35_percent,rent_25_to_30_percent,rent_20_to_25_percent,rent_15_to_20_percent,rent_10_to_15_percent,rent_under_10_percent,total_pop,male_pop,female_pop,median_age,white_pop,black_pop,asian_pop,hispanic_pop,amerindian_pop,other_race_pop,two_or_more_races_pop,not_hispanic_pop,commuters_by_public_transportation,households,median_income,income_per_capita,housing_units,vacant_housing_units,vacant_housing_units_for_rent,vacant_housing_units_for_sale,median_rent,percent_income_spent_on_rent,owner_occupied_housing_units,million_dollar_housing_units,mortgaged_housing_units,families_with_young_children,...,occupation_services,one_car,two_cars,three_cars,pop_25_64,pop_determined_poverty_status,population_1_year_and_over,population_3_years_over,poverty,sales_office_employed,some_college_and_associates_degree,walked_to_work,worked_at_home,workers_16_and_over,associates_degree,bachelors_degree,high_school_diploma,less_one_year_college,masters_degree,one_year_more_college,pop_25_years_over,commute_35_44_mins,commute_60_more_mins,commute_less_10_mins,commuters_16_over,hispanic_any_race,do_date,pop_5_years_over,speak_only_english_at_home,speak_spanish_at_home,speak_spanish_at_home_low_english,pop_15_and_over,pop_never_married,pop_now_married,pop_separated,pop_widowed,pop_divorced,year,income_dist,edu_dist
0,72015280202,663,959,1977.0,111,132,25,0,0,30,160,21,0,18,4476,2198,2278,38.0,11,0,0,4465,0,0,0,11,0,1622,14172.0,8110.0,2510,888,80,128,321.0,24.8,1125,26,420,401,...,191,1049,309,0,2133,4465,NaN,4258,2541,239,NaN,2,0,855,183,576,1001,43,100,261,2954,18,24,2,855,4465,20122016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,0.714839,0.207384
1,72031050902,488,1583,1977.0,235,106,0,44,0,0,0,40,31,19,5454,2374,3080,38.7,0,0,0,5421,0,33,0,33,87,2071,29691.0,13212.0,2372,301,28,29,501.0,38.4,1596,0,952,221,...,504,697,759,164,2748,5454,NaN,5381,1645,783,NaN,0,17,2098,495,769,828,53,185,425,3738,222,312,107,2081,5421,20122016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,0.380926,0.121093
2,72127002800,594,502,1959.0,140,88,63,49,9,109,17,54,0,29,2381,1158,1223,45.4,29,0,0,2352,0,0,0,29,227,1096,14810.0,9081.0,1475,379,113,13,337.0,30.0,538,0,142,100,...,364,410,133,17,1484,2381,NaN,2309,1231,326,NaN,170,0,1016,134,151,503,46,17,166,1846,62,19,59,1016,2352,20122016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,0.725598,0.268158
3,72127009121,515,1004,1968.0,107,146,60,0,32,49,33,63,28,0,3836,1585,2251,41.8,63,0,0,3773,0,0,0,63,30,1519,27355.0,17437.0,1861,342,34,17,581.0,40.1,1001,0,538,223,...,357,745,437,104,1839,3727,NaN,3709,1258,507,NaN,17,17,1556,454,650,361,82,331,264,2754,70,143,0,1539,3773,20122016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,0.334120,0.124323
4,72031050302,378,974,1964.0,138,103,9,0,48,34,51,43,0,9,3387,1538,1849,43.7,16,0,0,3371,0,0,0,16,68,1352,27562.0,12659.0,1693,341,94,32,513.0,31.2,917,19,460,93,...,266,616,387,115,1793,3344,NaN,3322,1001,385,NaN,31,17,1172,310,442,625,56,154,402,2469,166,75,118,1155,3371,20122016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,0.460297,0.124111


In [ ]:
#Adjusting for inflation
demo_data_2012['median_income'] = demo_data_2012['median_income']*1.09
demo_data_2017['median_income'] = demo_data_2017['median_income']*1.02
demo_data_2012['median_rent'] = demo_data_2012['median_rent']*1.09
demo_data_2017['median_rent'] = demo_data_2017['median_rent']*1.02
demo_data_2012['owner_occupied_housing_units_median_value'] = demo_data_2012['owner_occupied_housing_units_median_value']*1.09
demo_data_2017['owner_occupied_housing_units_median_value'] = demo_data_2017['owner_occupied_housing_units_median_value']*1.02

In [ ]:
#Concating Yearly Data to derive time-series variables
demo_data = pd.concat((demo_data_2011,demo_data_2012,demo_data_2017))
demo_data.head()

,geo_id,nonfamily_households,family_households,median_year_structure_built,rent_burden_not_computed,rent_over_50_percent,rent_40_to_50_percent,rent_35_to_40_percent,rent_30_to_35_percent,rent_25_to_30_percent,rent_20_to_25_percent,rent_15_to_20_percent,rent_10_to_15_percent,rent_under_10_percent,total_pop,male_pop,female_pop,median_age,white_pop,black_pop,asian_pop,hispanic_pop,amerindian_pop,other_race_pop,two_or_more_races_pop,not_hispanic_pop,commuters_by_public_transportation,households,median_income,income_per_capita,housing_units,vacant_housing_units,vacant_housing_units_for_rent,vacant_housing_units_for_sale,median_rent,percent_income_spent_on_rent,owner_occupied_housing_units,million_dollar_housing_units,mortgaged_housing_units,families_with_young_children,...,occupation_services,one_car,two_cars,three_cars,pop_25_64,pop_determined_poverty_status,population_1_year_and_over,population_3_years_over,poverty,sales_office_employed,some_college_and_associates_degree,walked_to_work,worked_at_home,workers_16_and_over,associates_degree,bachelors_degree,high_school_diploma,less_one_year_college,masters_degree,one_year_more_college,pop_25_years_over,commute_35_44_mins,commute_60_more_mins,commute_less_10_mins,commuters_16_over,hispanic_any_race,pop_5_years_over,speak_only_english_at_home,speak_spanish_at_home,speak_spanish_at_home_low_english,pop_15_and_over,pop_never_married,pop_now_married,pop_separated,pop_widowed,pop_divorced,do_date,year,income_dist,edu_dist
0,26011990100,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,...,0,0,0,0,0,0,0.0,0,0,0,0.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20072011,2011,0.0,NaN
1,48469980000,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,...,0,0,0,0,0,0,0.0,0,0,0,0.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20072011,2011,0.0,NaN
2,12009980000,0,0,NaN,0,0,0,0,0,0,0,0,0,0,32,32,0,22.6,32,0,0,0,0,0,0,32,0,0,NaN,24444.0,0,0,0,0,NaN,NaN,0,0,0,0,...,0,0,0,0,7,0,32.0,32,0,0,0.0,8,24,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,8,0,32.0,32.0,0.0,0.0,32.0,32.0,0.0,0.0,0.0,0.0,20072011,2011,0.0,NaN
3,4012980000,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,...,0,0,0,0,0,0,0.0,0,0,0,0.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20072011,2011,0.0,NaN
4,41015990101,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,0,...,0,0,0,0,0,0,0.0,0,0,0,0.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20072011,2011,0.0,NaN


In [ ]:
#Calculate Time Series Variables (difference of population, income, education over years)
def calc_time_series_vars(census_data,variable_list,start_yr,end_yr):
  
  df = demo_data_2017[['geo_id']]
  vars = variable_list
  year = [start_yr,end_yr]

  for j in vars:
      df['{}_{}_{}'.format(end_yr,start_yr,j)] = (census_data.loc[census_data['year'] == str(end_yr), j] - census_data.loc[census_data['year'] == str(start_yr), j])

  return df

In [ ]:
#Deriving race based features
demo_data['other_races'] = demo_data['black_pop']+ demo_data['asian_pop'] + demo_data['hispanic_pop'] + demo_data['amerindian_pop'] + demo_data['other_race_pop'] + demo_data['two_or_more_races_pop']

In [ ]:
#Calculating time changes for study variables
var_list = ['median_income','owner_occupied_housing_units_median_value','median_rent','income_dist','edu_dist','white_pop','other_races','commuters_by_public_transportation','income_per_capita','housing_units','vacant_housing_units','bachelors_degree']
df = calc_time_series_vars(demo_data,var_list,2012,2017)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
#Calculating time changes between 2011-2012
df2 = calc_time_series_vars(demo_data,var_list,2011,2012)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
#Visualising
df2.head()

,geo_id,2012_2011_median_income,2012_2011_owner_occupied_housing_units_median_value,2012_2011_median_rent,2012_2011_income_dist,2012_2011_edu_dist,2012_2011_white_pop,2012_2011_other_races,2012_2011_commuters_by_public_transportation,2012_2011_income_per_capita,2012_2011_housing_units,2012_2011_vacant_housing_units,2012_2011_bachelors_degree
0,72031050111,NaN,NaN,NaN,0.522534,NaN,51,208,0,NaN,57,0,NaN
1,72127010022,NaN,NaN,NaN,0.759207,NaN,217,988,34,NaN,623,140,NaN
2,72111730700,NaN,NaN,NaN,0.381772,NaN,765,1027,0,-12442.0,771,179,NaN
3,72113072201,NaN,NaN,NaN,0.000000,NaN,0,0,0,NaN,0,0,NaN
4,72113073005,NaN,NaN,NaN,0.000000,NaN,0,0,0,NaN,0,0,NaN


In [ ]:
#Merging study datasets
df = df.merge(df2,how='left',on='geo_id')
df.head()

,geo_id,2017_2012_median_income,2017_2012_owner_occupied_housing_units_median_value,2017_2012_median_rent,2017_2012_income_dist,2017_2012_edu_dist,2017_2012_white_pop,2017_2012_other_races,2017_2012_commuters_by_public_transportation,2017_2012_income_per_capita,2017_2012_housing_units,2017_2012_vacant_housing_units,2017_2012_bachelors_degree,2012_2011_median_income,2012_2011_owner_occupied_housing_units_median_value,2012_2011_median_rent,2012_2011_income_dist,2012_2011_edu_dist,2012_2011_white_pop,2012_2011_other_races,2012_2011_commuters_by_public_transportation,2012_2011_income_per_capita,2012_2011_housing_units,2012_2011_vacant_housing_units,2012_2011_bachelors_degree
0,72031050111,16843.07,NaN,-246.20,-0.452234,-0.180913,19,890,0,25400.0,1179,651,323.0,NaN,NaN,NaN,0.522534,NaN,51,208,0,NaN,57,0,NaN
1,72127010022,43426.47,179444.0,147.70,-0.638473,-0.033616,-178,1838,2,22325.0,884,234,808.0,NaN,NaN,NaN,0.759207,NaN,217,988,34,NaN,623,140,NaN
2,72111730700,-16875.06,36802.0,-238.02,0.201064,-0.024530,-797,3076,0,-3736.0,751,122,441.0,NaN,NaN,NaN,0.381772,NaN,765,1027,0,-12442.0,771,179,NaN
3,72113072201,NaN,NaN,NaN,0.641039,0.124669,0,3846,0,NaN,1498,159,458.0,NaN,NaN,NaN,0.000000,NaN,0,0,0,NaN,0,0,NaN
4,72113073005,NaN,NaN,NaN,0.336173,0.135124,0,2822,0,NaN,1058,132,568.0,NaN,NaN,NaN,0.000000,NaN,0,0,0,NaN,0,0,NaN


In [ ]:
#Concating SVI Variables
svi_data = pd.concat((svi_2000,svi_2010,svi_2014,svi_2016,svi_2018))
svi_data.head()

,STATE_FIPS,CNTY_FIPS,STCOFIPS,TRACT,FIPS,STATE_NAME,STATE_ABBR,COUNTY,G1V1R,G1V2R,G1V3R,G1V4R,G2V1R,G2V2R,G2V3R,G2V4R,G3V1R,G3V2R,G4V1R,G4V2R,G4V3R,G4V4R,G4V5R,NYG1V1P,NYG1V2P,NYG1V3P,NYG1V4P,NYG1TP,NYG2V1P,NYG2V2P,NYG2V3P,NYG2V4P,NYG2TP,NYG3V1P,NYG3V2P,NYG3TP,NYG4V1P,NYG4V2P,NYG4V3P,NYG4V4P,...,SPL_THEME2,RPL_THEME2,EPL_MINRTY,EPL_LIMENG,SPL_THEME3,RPL_THEME3,EPL_MUNIT,EPL_MOBILE,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,F_POV,F_UNEMP,F_PCI,F_NOHSDP,F_THEME1,F_AGE65,F_AGE17,F_DISABL,F_SNGPNT,F_THEME2,F_MINRTY,F_LIMENG,F_THEME3,F_MUNIT,F_MOBILE,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP
0,36.0,29.0,36029.0,4401.0,36029004401,New York,NY,Erie,0.2079,0.0929,15730.0,0.2425,0.0942,0.289283,0.214371,0.203782,0.855139,0.001903,0.014554,0.0,0.0224,0.2217,0.000877,0.7418,0.7311,0.7090,0.630,0.785,0.2864,0.7809,0.5599,0.8621,0.7571,0.7938,0.0614,0.4123,0.3152,0.0,0.4028,0.5491,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,36.0,29.0,36029.0,4402.0,36029004402,New York,NY,Erie,0.5162,0.1501,7237.0,0.0903,0.0487,0.442105,0.308320,0.500903,0.979649,0.008157,0.034510,0.0,0.0658,0.5320,0.000351,0.9799,0.8787,0.9812,0.156,0.844,0.0494,0.9935,0.8863,0.9985,0.9329,0.9060,0.2535,0.5613,0.4189,0.0,0.6125,0.7739,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,36.0,29.0,36029.0,4500.0,36029004500,New York,NY,Erie,0.1599,0.0346,23050.0,0.1402,0.1665,0.224554,0.202743,0.129698,0.281193,0.013495,0.128056,0.0,0.0177,0.1712,0.071797,0.6450,0.2415,0.3354,0.318,0.341,0.7613,0.3272,0.5011,0.7255,0.6824,0.5344,0.3709,0.4392,0.6081,0.0,0.3419,0.4982,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,36.0,29.0,36029.0,4601.0,36029004601,New York,NY,Erie,0.2489,0.0247,17744.0,0.2101,0.0985,0.187429,0.144658,0.101551,0.260286,0.014106,0.035146,0.0,0.0184,0.1274,0.000000,0.7998,0.1107,0.5885,0.542,0.521,0.3160,0.1660,0.1991,0.6338,0.1181,0.5187,0.3810,0.4369,0.4226,0.0,0.3523,0.4392,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,36.0,29.0,36029.0,4602.0,36029004602,New York,NY,Erie,0.0000,1.0000,4706.0,0.0228,0.0527,0.000000,0.007608,0.000000,0.184251,0.110092,0.000000,0.0,0.0000,0.0000,1.000000,0.0000,0.9981,0.9943,0.015,0.510,0.0598,0.0000,0.0045,0.0000,0.0047,0.4473,0.7981,0.6125,0.0000,0.0,0.0000,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Calculate Time Series Variables (difference of population, income, education over years)
def calc_time_series_vars_svi(census_data,variable_list,start_yr,end_yr):
  
  df = svi_2016[['FIPS']]
  vars = variable_list
  year = [start_yr,end_yr]

  for j in vars:
      df['{}_{}_{}'.format(end_yr,start_yr,j)] = (census_data.loc[census_data['year'] == str(end_yr), j] - census_data.loc[census_data['year'] == str(start_yr), j])

  return df

In [ ]:
#Doing the same changes to social vulnerability datasets
svi_var_list = ['EP_POV','EP_UNEMP','EP_SNGPNT','EP_MINRTY','EP_MUNIT','EP_MOBILE','EP_CROWD']
df3 = calc_time_series_vars_svi(svi_data,svi_var_list,2014,2018)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
#Final Merges
df3 = df3.rename(columns={'FIPS':'geo_id'})
df = df.merge(df3,how='left',on='geo_id')
df.head()

,geo_id,2017_2012_median_income,2017_2012_owner_occupied_housing_units_median_value,2017_2012_median_rent,2017_2012_income_dist,2017_2012_edu_dist,2017_2012_white_pop,2017_2012_other_races,2017_2012_commuters_by_public_transportation,2017_2012_income_per_capita,2017_2012_housing_units,2017_2012_vacant_housing_units,2017_2012_bachelors_degree,2012_2011_median_income,2012_2011_owner_occupied_housing_units_median_value,2012_2011_median_rent,2012_2011_income_dist,2012_2011_edu_dist,2012_2011_white_pop,2012_2011_other_races,2012_2011_commuters_by_public_transportation,2012_2011_income_per_capita,2012_2011_housing_units,2012_2011_vacant_housing_units,2012_2011_bachelors_degree,2018_2014_EP_POV,2018_2014_EP_UNEMP,2018_2014_EP_SNGPNT,2018_2014_EP_MINRTY,2018_2014_EP_MUNIT,2018_2014_EP_MOBILE,2018_2014_EP_CROWD
0,72031050111,16843.07,NaN,-246.20,-0.452234,-0.180913,19,890,0,25400.0,1179,651,323.0,NaN,NaN,NaN,0.522534,NaN,51,208,0,NaN,57,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,72127010022,43426.47,179444.0,147.70,-0.638473,-0.033616,-178,1838,2,22325.0,884,234,808.0,NaN,NaN,NaN,0.759207,NaN,217,988,34,NaN,623,140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,72111730700,-16875.06,36802.0,-238.02,0.201064,-0.024530,-797,3076,0,-3736.0,751,122,441.0,NaN,NaN,NaN,0.381772,NaN,765,1027,0,-12442.0,771,179,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,72113072201,NaN,NaN,NaN,0.641039,0.124669,0,3846,0,NaN,1498,159,458.0,NaN,NaN,NaN,0.000000,NaN,0,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,72113073005,NaN,NaN,NaN,0.336173,0.135124,0,2822,0,NaN,1058,132,568.0,NaN,NaN,NaN,0.000000,NaN,0,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
NYC_COUNTIES = [
    "005", # Bronx
    "047", # Kings (Brooklyn)
    "061", # New York County (Manhattan)
    "081", # Queens
    "085", # Richmond (Staten Island)
]

In [ ]:
spatial_data = gpd.read_file('tl_2019_36_tract.shp')
spatial_data['GEOID'] = spatial_data['GEOID'].astype(int)
spatial_data= spatial_data[spatial_data['COUNTYFP'].isin(NYC_COUNTIES)]
spatial_data.head()

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,36,081,044800,36081044800,448,Census Tract 448,G5020,S,208002,0,+40.7110219,-073.8026344,"POLYGON ((-73.80646 40.71206, -73.80556 40.712..."
1,36,081,045800,36081045800,458,Census Tract 458,G5020,S,245281,0,+40.7152626,-073.7909261,"POLYGON ((-73.79364 40.71382, -73.79362 40.713..."
2,36,081,046200,36081046200,462,Census Tract 462,G5020,S,249611,0,+40.7098547,-073.7879749,"POLYGON ((-73.79203 40.71107, -73.79101 40.711..."
3,36,081,046300,36081046300,463,Census Tract 463,G5020,S,159415,0,+40.7440007,-073.8710900,"POLYGON ((-73.87468 40.74335, -73.87423 40.743..."
4,36,081,046400,36081046400,464,Census Tract 464,G5020,S,228767,0,+40.7168637,-073.7869958,"POLYGON ((-73.79187 40.71379, -73.79085 40.714..."


In [ ]:
#Calculating queen weights for each GeoID (census tract)
w = Queen.from_dataframe(spatial_data,idVariable='GEOID')
w.transform = 'R'

('WARNING: ', 36061000100, ' is an island (no neighbors)')


/usr/local/lib/python3.6/dist-packages/libpysal/weights/weights.py:172: UserWarning:

The weights matrix is not fully connected: 
 There are 2 disconnected components.
 There is 1 island with id: 36061000100.



In [ ]:
spatial_data = spatial_data.merge(df, how='inner',left_on='GEOID',right_on='geo_id')
spatial_data.head()

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,geo_id,2017_2012_median_income,2017_2012_owner_occupied_housing_units_median_value,2017_2012_median_rent,2017_2012_income_dist,2017_2012_edu_dist,2017_2012_white_pop,2017_2012_other_races,2017_2012_commuters_by_public_transportation,2017_2012_income_per_capita,2017_2012_housing_units,2017_2012_vacant_housing_units,2017_2012_bachelors_degree,2012_2011_median_income,2012_2011_owner_occupied_housing_units_median_value,2012_2011_median_rent,2012_2011_income_dist,2012_2011_edu_dist,2012_2011_white_pop,2012_2011_other_races,2012_2011_commuters_by_public_transportation,2012_2011_income_per_capita,2012_2011_housing_units,2012_2011_vacant_housing_units,2012_2011_bachelors_degree,2018_2014_EP_POV,2018_2014_EP_UNEMP,2018_2014_EP_SNGPNT,2018_2014_EP_MINRTY,2018_2014_EP_MUNIT,2018_2014_EP_MOBILE,2018_2014_EP_CROWD
0,36,081,044800,36081044800,448,Census Tract 448,G5020,S,208002,0,+40.7110219,-073.8026344,"POLYGON ((-73.80646 40.71206, -73.80556 40.712...",36081044800,-11203.74,495102.0,635.70,0.037868,-0.083222,-3409,2581,670,-8190.0,-778,31,-73.0,-7063.26,-26378.0,NaN,-0.435488,NaN,3215,-1636,-30,76.0,846,38,NaN,-17.1,-23.5,2.6,-93.3,-82.3,0.0,-21.8
1,36,081,045800,36081045800,458,Census Tract 458,G5020,S,245281,0,+40.7152626,-073.7909261,"POLYGON ((-73.79364 40.71382, -73.79362 40.713...",36081045800,-8801.66,465607.0,625.05,-0.096485,-0.078431,-4580,1979,646,731.0,-1350,-67,143.0,42186.78,95493.0,380.35,-0.069210,NaN,610,-39,3,10250.0,-314,-323,NaN,-16.9,-11.9,-3.2,-94.4,-91.3,31.4,-23.0
2,36,081,046200,36081046200,462,Census Tract 462,G5020,S,249611,0,+40.7098547,-073.7879749,"POLYGON ((-73.79203 40.71107, -73.79101 40.711...",36081046200,16139.15,391545.0,773.57,-0.184451,0.053082,-3183,5856,2035,-5151.0,-613,-166,494.0,-5846.39,-17271.0,187.93,0.124673,NaN,-5605,1664,132,2394.0,-1033,63,NaN,-4.4,-7.1,-4.1,-85.5,-87.4,0.0,-16.9
3,36,081,046300,36081046300,463,Census Tract 463,G5020,S,159415,0,+40.7440007,-073.8710900,"POLYGON ((-73.87468 40.74335, -73.87423 40.743...",36081046300,-8269.56,311926.0,221.94,-0.083255,-0.011053,-1053,1140,983,-2212.0,-401,-93,140.0,24195.76,182994.0,466.52,-0.119574,NaN,1041,-1237,211,9697.0,383,117,NaN,4.9,0.7,3.4,-60.9,-78.0,1.2,-14.0
4,36,081,046400,36081046400,464,Census Tract 464,G5020,S,228767,0,+40.7168637,-073.7869958,"POLYGON ((-73.79187 40.71379, -73.79085 40.714...",36081046400,22743.78,519311.0,831.68,0.148496,0.027378,-4709,807,391,558.0,-2161,-126,-863.0,4594.38,-107959.0,-389.90,0.011567,NaN,3374,288,20,4391.0,1680,-3,NaN,1.6,-2.1,6.2,16.1,0.5,1.5,2.5


In [ ]:
#Dropping education since all values are null
spatial_data = spatial_data.drop(columns='2012_2011_edu_dist')
spatial_data = spatial_data.drop(columns='2012_2011_bachelors_degree')

In [ ]:
#Fill NAs with 0 for spatial lag calculations
spatial_data = spatial_data.fillna(999)

In [ ]:
gentrification_data = pd.read_csv('/content/gentrification_13_18.csv')
gentrification_data = gentrification_data.drop(columns=['Unnamed: 0'])
gentrification_data = gentrification_data.rename(columns={'geo_id':'GEOID'})
gentrification_data.head()

,GEOID,total_pop_18,white_pop_18,black_pop_18,asian_pop_18,hispanic_pop_18,amerindian_pop_18,other_race_pop_18,two_or_more_races_pop_18,median_income_18,median_home_value_18,median_rent_18,bachelors_degree_18,total_pop_13,white_pop_13,black_pop_13,asian_pop_13,hispanic_pop_13,amerindian_pop_13,other_race_pop_13,two_or_more_races_pop_13,median_income_13,median_rent_13,median_home_value_13,bachelors_degree_13,total_pop_10,white_pop_10,black_pop_10,asian_pop_10,hispanic_pop_10,amerindian_pop_10,other_race_pop_10,two_or_more_races_pop_10,median_income_10,median_rent_10,median_home_value_10,bachelors_degree_10,total_population00,median_income00,median_home_value00,educational_attainment_00,white_alone00,black_alone00,native00,asian00,native_hawaiian_pacific_islander00,hispanic_or_latino00,educational_attainment_pct,educational_attainment_pct13,educational_attainment_change,home_pct_change,has_pop_500+_2013,has_pop_500+_2018,median_40th_home_2013,median_40th_income_2013,educational_is_upper3rd,median_home_higher,median_home_top_third,gentrified,pct_white_alone_change,pct_white_alone_2018,pct_black_alone_change,pct_black_alone_2018,pct_asian_alone_change,pct_asian_alone_2018,pct_hispanic_alone_change,pct_hispanic_alone_2018,low_population,eligible_for_gentrification,not_eligible_highpop
0,36005050400,222,0,25,3,194,0,0,0,NaN,NaN,725.0,0,33.0,14.0,7.0,0.0,12.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,5.000000,NaN,0.0000,0.000000,2.000000,0.000000,0.0,0.000000,0.0,3.000000,0.000000,0.000000,0.000000,NaN,False,False,False,False,False,False,False,False,-42.424242,0.000000,-9.950860,11.261261,1.351351,1.351351,51.023751,87.387387,True,False,False
1,36081005000,101,9,0,0,92,0,0,0,NaN,NaN,1571.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,103.457771,27411.00086,224999.9975,23.647491,10.838433,5.911873,0.0,30.544676,0.0,56.162788,0.000000,NaN,NaN,NaN,False,False,False,False,False,False,False,False,NaN,8.910891,NaN,0.000000,NaN,0.000000,NaN,91.089109,True,False,False
2,36005043500,361,15,153,0,87,101,0,5,NaN,NaN,870.0,10,16.0,0.0,16.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,77.0,0.0,21.0,0.0,56.0,0.0,0.0,0.0,17604.0,NaN,312500.0,NaN,65.437142,80487.99663,225000.0057,0.000000,2.845093,18.967287,0.0,0.000000,0.0,36.986210,2.770083,0.000000,2.770083,NaN,False,False,False,False,False,False,False,False,4.155125,4.155125,-57.617729,42.382271,0.000000,0.000000,24.099723,24.099723,True,False,False
3,36081056100,60,35,0,17,8,0,0,0,100000.0,425000.0,NaN,12,146.0,21.0,40.0,20.0,55.0,0.0,0.0,10.0,112000.0,1750.0,312500.0,39.0,55.0,9.0,0.0,0.0,46.0,0.0,0.0,0.0,104875.0,1238.0,NaN,NaN,7.000000,NaN,NaN,0.000000,0.000000,0.000000,0.0,0.000000,0.0,7.000000,20.000000,26.712329,-6.712329,0.280000,False,False,True,False,False,True,True,False,43.949772,58.333333,-27.397260,0.000000,14.634703,28.333333,-24.337900,13.333333,True,False,False
4,36081011400,1382,176,209,341,627,0,4,25,49242.0,591900.0,1305.0,17,883.0,137.0,67.0,33.0,583.0,10.0,0.0,53.0,44464.0,1099.0,448700.0,140.0,984.0,82.0,49.0,148.0,705.0,0.0,0.0,0.0,50556.0,1026.0,458500.0,NaN,1120.000000,28571.00000,246400.0000,86.000000,358.000000,140.000000,13.0,105.000000,0.0,464.000000,1.230101,15.855040,-14.624938,0.239144,True,True,False,True,False,True,True,False,-2.780122,12.735166,7.535241,15.123010,20.937126,24.674385,-20.655885,45.369030,False,False,True


In [ ]:
spatial_data = spatial_data.merge(gentrification_data[['GEOID','gentrified']], on = 'GEOID',how='left')

In [ ]:
def f(row):
    if row['gentrified'] == False:
      val = 0
    elif row['gentrified'] == True:
      val = 1
    else:
      val = 2

    return val

spatial_data['gentrification_label'] = spatial_data.apply(f, axis=1)

In [ ]:
#Calculating spatial weights
import libpysal as lps

spatial_data['lagged_spatial_income'] = lps.weights.lag_spatial(w, spatial_data['2012_2011_median_income'])
spatial_data['lagged_spatial_rent'] = lps.weights.lag_spatial(w, spatial_data['2012_2011_median_rent'])
spatial_data['lagged_income_dist'] = lps.weights.lag_spatial(w, spatial_data['2012_2011_income_dist'])
spatial_data['lagged_white_pop'] = lps.weights.lag_spatial(w, spatial_data['2012_2011_white_pop'])
spatial_data['lagged_other_races'] = lps.weights.lag_spatial(w, spatial_data['2012_2011_other_races'])
#spatial_data['lagged_edu_dist'] = lps.weights.lag_spatial(w, spatial_data['2012_2011_edu_dist'])
spatial_data['lagged_public_commute'] = lps.weights.lag_spatial(w, spatial_data['2012_2011_commuters_by_public_transportation'])
spatial_data['lagged_income_per_capita'] = lps.weights.lag_spatial(w, spatial_data['2012_2011_income_per_capita'])
spatial_data['lagged_housing_units'] = lps.weights.lag_spatial(w, spatial_data['2012_2011_housing_units'])
spatial_data['lagged_vacant_housing_units'] = lps.weights.lag_spatial(w, spatial_data['2012_2011_vacant_housing_units'])

In [ ]:
spatial_data['gentrification_label'].value_counts()

0    2122
1      45
Name: gentrification_label, dtype: int64

In [ ]:
#Running models as is without balancing the datasets to include equal proportions of each classes for better training setting
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
#from sklearn.metrics import roc_auc_ovr
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn import preprocessing

spatial_data_training = spatial_data

features = ['2017_2012_median_income','2017_2012_median_rent','2017_2012_income_dist',
            '2017_2012_edu_dist','2017_2012_white_pop','2017_2012_other_races','2017_2012_commuters_by_public_transportation',
            '2017_2012_income_per_capita','2017_2012_housing_units','2017_2012_vacant_housing_units','2017_2012_bachelors_degree',
            '2018_2014_EP_POV','2018_2014_EP_UNEMP','2018_2014_EP_SNGPNT','2018_2014_EP_MINRTY','2018_2014_EP_MUNIT','2018_2014_EP_MOBILE',
            '2018_2014_EP_CROWD','lagged_spatial_income','lagged_spatial_rent','lagged_income_dist','lagged_white_pop',
            'lagged_other_races','lagged_public_commute','lagged_income_per_capita','lagged_housing_units','lagged_vacant_housing_units']


X = spatial_data_training[features]
y = spatial_data_training['gentrification_label']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 999)


mu=X_train.iloc[:,:].mean()
sigma=X_train.iloc[::].std()

X_train.iloc[:,:]=(X_train.iloc[:,:]-mu)/sigma
X_test.iloc[:,:]=(X_test.iloc[:,:]-mu)/sigma

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
from imblearn.over_sampling import SMOTE

os = SMOTE(random_state=0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
columns = X_train.columns

mu=X_train.iloc[:,:].mean()
sigma=X_train.iloc[::].std()

X_train.iloc[:,:]=(X_train.iloc[:,:]-mu)/sigma
X_test.iloc[:,:]=(X_test.iloc[:,:]-mu)/sigma


os_data_X,os_data_y = os.fit_sample(X_train, y_train)

os_data_X = pd.DataFrame(data=os_data_X,columns=columns)

os_data_y= pd.DataFrame(data=os_data_y,columns=['y'])

# we can Check the numbers of our data
print("length of oversampled data is ",len(os_data_X))
print("Number of no subscription in oversampled data",len(os_data_y[os_data_y['y']==0]))
print("Number of subscription",len(os_data_y[os_data_y['y']==1]))
print("Proportion of no subscription data in oversampled data is ",len(os_data_y[os_data_y['y']==0])/len(os_data_X))
print("Proportion of subscription data in oversampled data is ",len(os_data_y[os_data_y['y']==1])/len(os_data_X))

length of oversampled data is  2956
Number of no subscription in oversampled data 1478
Number of subscription 1478
Proportion of no subscription data in oversampled data is  0.5
Proportion of subscription data in oversampled data is  0.5


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning:

The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
rfe = RFE(logreg, 20)
rfe = rfe.fit(os_data_X, os_data_y.values.ravel())
print(rfe.support_)
print(rfe.ranking_)

[ True False  True  True False  True  True  True  True  True  True  True
  True False  True  True  True False  True  True  True False False  True
  True  True False]
[1 8 1 1 2 1 1 1 1 1 1 1 1 4 1 1 1 5 1 1 1 7 6 1 1 1 3]


In [ ]:
X_train

,2017_2012_median_income,2017_2012_median_rent,2017_2012_income_dist,2017_2012_edu_dist,2017_2012_white_pop,2017_2012_other_races,2017_2012_commuters_by_public_transportation,2017_2012_income_per_capita,2017_2012_housing_units,2017_2012_vacant_housing_units,2017_2012_bachelors_degree,2018_2014_EP_POV,2018_2014_EP_UNEMP,2018_2014_EP_SNGPNT,2018_2014_EP_MINRTY,2018_2014_EP_MUNIT,2018_2014_EP_MOBILE,2018_2014_EP_CROWD,lagged_spatial_income,lagged_spatial_rent,lagged_income_dist,lagged_white_pop,lagged_other_races,lagged_public_commute,lagged_income_per_capita,lagged_housing_units,lagged_vacant_housing_units
1584,-0.126063,-0.064830,-0.238547,0.404306,1.038146,-0.178151,-0.139165,-0.067441,0.682794,0.404343,0.796061,0.089767,0.037685,-0.362858,-0.223636,0.050089,0.123856,-0.310912,0.578386,-0.480644,-0.586763,0.734663,-0.238007,0.142220,0.824194,0.504690,0.684501
841,0.814999,1.264471,1.112115,-0.517496,-0.924075,0.399200,0.191082,0.367036,-0.672107,-0.358020,0.638036,0.065896,0.050973,-0.337327,-0.260778,0.075641,0.123073,-0.111503,0.085110,0.453002,2.526144,-0.277023,-1.660874,-1.530767,-0.230948,-1.817108,-0.117301
905,-0.498732,-0.423730,0.365160,0.672244,-0.327770,1.029576,0.524931,-0.232835,0.669039,0.870087,0.022284,0.125959,0.143994,0.238926,-0.084843,-0.067022,0.123073,0.128525,0.036118,-0.562704,-1.667566,0.537489,0.693942,-0.294534,-0.839170,-0.404448,-1.623540
728,2.341076,1.845602,2.051014,2.138409,0.563402,-1.236071,1.086950,3.464088,0.338911,0.349703,1.831396,0.115948,0.131487,-0.184146,-0.700614,-0.030824,0.123073,-0.288755,-1.869941,0.554994,1.149764,0.247514,0.349243,-0.292683,-2.022532,0.366312,-0.103170
40,-0.599478,-0.056898,-0.324299,-0.269897,-0.673150,-0.663541,-0.570286,-0.099444,-0.996734,-0.207108,-1.673760,0.062816,0.145557,-0.268031,-0.057475,0.131003,-0.158052,-0.008106,1.526988,1.198794,0.721027,-0.218147,-0.107986,0.180466,1.064459,-0.533237,-0.761842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033,-0.781334,-0.906576,-1.134801,-0.106057,1.103854,-0.973538,0.856378,0.046969,0.481966,0.607293,0.488185,0.037405,0.091621,-0.264384,-1.435630,-0.011661,0.148131,-0.251828,1.688948,2.621507,1.552073,-0.280109,0.137941,-0.880819,0.571153,-0.621478,-0.247303
1731,-3.229284,0.778701,-3.049719,-2.363604,0.866893,-1.303558,-1.050645,-0.230341,0.488844,0.976766,-0.830506,0.158300,0.093184,0.300928,-0.106346,-0.030824,0.123073,-0.281370,0.647211,0.950275,0.967476,0.796924,-0.341748,-0.065053,0.988324,0.652062,0.022791
763,-1.570601,-1.846639,0.486916,0.041569,-0.018940,-1.082556,-1.110689,-0.284630,0.027352,0.578672,-0.429995,0.159070,0.107255,0.381166,1.412557,1.242497,0.123073,0.220844,0.428571,0.091366,-0.251483,1.776732,-1.402089,-0.295767,0.032366,-0.028950,-0.183852
835,-0.129098,-0.184985,-0.059043,-0.057972,-1.387730,-0.368747,0.063787,0.628085,-0.598516,0.219607,-0.168436,0.138279,0.131487,-0.074731,-0.391751,-0.030824,0.123073,-0.056112,-0.642407,1.069154,0.390490,-2.180281,1.223831,0.303843,-1.213093,-1.358765,0.668547


In [ ]:
cols=['2017_2012_median_income', '2017_2012_white_pop','2017_2012_income_dist','2017_2012_edu_dist', '2017_2012_other_races', '2017_2012_commuters_by_public_transportation','2017_2012_income_per_capita', '2017_2012_housing_units', '2017_2012_vacant_housing_units', 
      '2017_2012_bachelors_degree', '2018_2014_EP_POV', '2018_2014_EP_UNEMP', '2018_2014_EP_SNGPNT', '2018_2014_EP_MUNIT', '2018_2014_EP_MOBILE','2018_2014_EP_CROWD','lagged_spatial_income','lagged_spatial_rent', 'lagged_income_dist','lagged_housing_units']

Xn=os_data_X[cols]
yn=os_data_y['y']

In [ ]:
import statsmodels.api as sm
logit_model=sm.Logit(yn,Xn)
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.536866
         Iterations 6
                                       Results: Logit
Model:                        Logit                    Pseudo R-squared:         0.225      
Dependent Variable:           y                        AIC:                      3213.9518  
Date:                         2020-07-15 04:06         BIC:                      3333.7836  
No. Observations:             2956                     Log-Likelihood:           -1587.0    
Df Model:                     19                       LL-Null:                  -2048.9    
Df Residuals:                 2936                     LLR p-value:              8.9297e-184
Converged:                    1.0000                   Scale:                    1.0000     
No. Iterations:               6.0000                                                        
--------------------------------------------------------------------------------------------
       

In [ ]:
cols = ['2017_2012_median_income','2017_2012_income_dist','2017_2012_edu_dist','2017_2012_other_races','2017_2012_commuters_by_public_transportation',
        '2017_2012_income_per_capita','2017_2012_housing_units','2017_2012_vacant_housing_units','2017_2012_bachelors_degree',
        '2018_2014_EP_POV','2018_2014_EP_MUNIT','2018_2014_EP_MOBILE','lagged_spatial_income',
        'lagged_spatial_rent','lagged_income_dist']

Xn=os_data_X[cols]
yn=os_data_y['y']
logit_model=sm.Logit(yn,Xn)
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.541793
         Iterations 6
                                       Results: Logit
Model:                       Logit                     Pseudo R-squared:          0.218      
Dependent Variable:          y                         AIC:                       3233.0780  
Date:                        2020-07-15 04:06          BIC:                       3322.9519  
No. Observations:            2956                      Log-Likelihood:            -1601.5    
Df Model:                    14                        LL-Null:                   -2048.9    
Df Residuals:                2941                      LLR p-value:               5.5925e-182
Converged:                   1.0000                    Scale:                     1.0000     
No. Iterations:              6.0000                                                          
--------------------------------------------------------------------------------------------

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
Xn_train, Xn_test, yn_train, yn_test = train_test_split(Xn, yn, test_size=0.3, random_state=0)
logreg = LogisticRegression()
logreg.fit(Xn_train, yn_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
y_pred = logreg.predict(Xn_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(Xn_test, yn_test)))

Accuracy of logistic regression classifier on test set: 0.81


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(yn_test, y_pred)
print(confusion_matrix)

[[364 103]
 [ 69 351]]


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(yn_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.78      0.81       467
           1       0.77      0.84      0.80       420

    accuracy                           0.81       887
   macro avg       0.81      0.81      0.81       887
weighted avg       0.81      0.81      0.81       887



In [ ]:
spatial_data.iloc[:,14:]

,2017_2012_median_income,2017_2012_owner_occupied_housing_units_median_value,2017_2012_median_rent,2017_2012_income_dist,2017_2012_edu_dist,2017_2012_white_pop,2017_2012_other_races,2017_2012_commuters_by_public_transportation,2017_2012_income_per_capita,2017_2012_housing_units,2017_2012_vacant_housing_units,2017_2012_bachelors_degree,2012_2011_median_income,2012_2011_owner_occupied_housing_units_median_value,2012_2011_median_rent,2012_2011_income_dist,2012_2011_white_pop,2012_2011_other_races,2012_2011_commuters_by_public_transportation,2012_2011_income_per_capita,2012_2011_housing_units,2012_2011_vacant_housing_units,2018_2014_EP_POV,2018_2014_EP_UNEMP,2018_2014_EP_SNGPNT,2018_2014_EP_MINRTY,2018_2014_EP_MUNIT,2018_2014_EP_MOBILE,2018_2014_EP_CROWD,gentrified,gentrification_label,lagged_spatial_income,lagged_home_value,lagged_spatial_rent,lagged_income_dist,lagged_white_pop,lagged_other_races,lagged_public_commute,lagged_income_per_capita,lagged_housing_units,lagged_vacant_housing_units
0,-11203.74,495102.0,635.70,0.037868,-0.083222,-3409,2581,670,-8190.0,-778,31,-73.0,-7063.26,-26378.0,999.00,-0.435488,3215,-1636,-30,76.0,846,38,-17.1,-23.5,2.6,-93.3,-82.3,0.0,-21.8,False,0,-32822.854000,-105452.782000,-299.944000,-0.236056,-62.200000,-561.000000,-104.600000,-9491.200000,480.600000,380.800000
1,-8801.66,465607.0,625.05,-0.096485,-0.078431,-4580,1979,646,731.0,-1350,-67,143.0,42186.78,95493.0,380.35,-0.069210,610,-39,3,10250.0,-314,-323,-16.9,-11.9,-3.2,-94.4,-91.3,31.4,-23.0,False,0,-9065.913333,-77619.500000,57.618333,-0.076920,-105.666667,-158.833333,27.333333,-4337.833333,57.333333,20.666667
2,16139.15,391545.0,773.57,-0.184451,0.053082,-3183,5856,2035,-5151.0,-613,-166,494.0,-5846.39,-17271.0,187.93,0.124673,-5605,1664,132,2394.0,-1033,63,-4.4,-7.1,-4.1,-85.5,-87.4,0.0,-16.9,True,1,-4140.120000,-11113.857143,107.161429,-0.127328,-912.142857,-472.714286,11.285714,-3174.000000,-493.714286,-41.428571
3,-8269.56,311926.0,221.94,-0.083255,-0.011053,-1053,1140,983,-2212.0,-401,-93,140.0,24195.76,182994.0,466.52,-0.119574,1041,-1237,211,9697.0,383,117,4.9,0.7,3.4,-60.9,-78.0,1.2,-14.0,False,0,-10884.158571,-21295.714286,326.705714,-0.109194,-808.857143,84.571429,33.714286,-5153.428571,-176.285714,11.571429
4,22743.78,519311.0,831.68,0.148496,0.027378,-4709,807,391,558.0,-2161,-126,-863.0,4594.38,-107959.0,-389.90,0.011567,3374,288,20,4391.0,1680,-3,1.6,-2.1,6.2,16.1,0.5,1.5,2.5,False,0,26428.710000,106350.227143,197.973571,0.016150,458.571429,-139.428571,104.785714,18682.857143,330.500000,83.857143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2162,8334.77,196695.0,956.45,0.019043,-0.057887,-5209,4807,1993,8252.0,-207,-66,636.0,983.73,33743.0,-36.75,-0.077862,4673,367,-39,189.0,1956,168,-16.5,-2.3,-3.7,-41.3,-51.5,1.0,-4.6,False,0,50960.004000,162178.800000,640.528000,0.125059,1264.400000,-546.800000,-107.000000,11736.200000,-209.400000,-79.000000
2163,-46269.17,-46797.0,-799.90,0.025759,-0.115664,-2377,1679,556,-16560.0,-328,35,-450.0,88641.17,-258097.0,1115.90,0.198431,2295,-2545,-747,23396.0,-514,-85,0.2,0.8,2.3,-15.9,-92.3,18.6,1.4,False,0,6838.228000,47507.400000,118.398000,-0.123647,1320.400000,646.200000,31.200000,-1606.000000,551.200000,10.800000
2164,13181.24,49128.0,797.83,-0.180557,-0.056462,619,1531,1892,14697.0,1571,282,586.0,-1959.72,82580.0,94.41,0.122839,-1139,1456,13,-13468.0,-506,-89,-15.6,-8.8,-10.3,-83.3,-85.7,20.3,-7.7,False,0,30952.075000,-34220.750000,412.557500,-0.116195,1340.750000,-284.250000,-141.250000,7161.250000,198.000000,-27.500000
2165,20240.69,173223.0,533.25,0.336203,0.028264,-3181,555,182,5720.0,-1323,-138,-501.0,40128.59,82063.0,439.71,-0.507538,169,-431,169,10112.0,-635,-275,-5.1,-9.5,-2.3,-24.9,-84.5,5.7,-4.0,False,0,29931.758333,102532.833333,390.333333,0.248413,458.000000,570.833333,-131.833333,1491.500000,-232.333333,-49.166667


In [ ]:
spatial_data_rh.head()

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,geo_id,2022_2017_median_income,2022_2017_median_rent,2022_2017_income_dist,2022_2017_edu_dist,2022_2017_other_races,2022_2017_commuters_by_public_transportation,2022_2017_income_per_capita,2022_2017_housing_units,2022_2017_vacant_housing_units,2022_2017_bachelors_degree,2017_2016_median_income,2017_2016_median_rent,2017_2016_income_dist,2017_2016_edu_dist,2017_2016_other_races,2017_2016_commuters_by_public_transportation,2017_2016_income_per_capita,2017_2016_housing_units,2017_2016_vacant_housing_units,2017_2016_bachelors_degree,2022_2018_EP_POV,2022_2018_EP_MUNIT,2022_2018_EP_MOBILE,lagged_spatial_income,lagged_income_dist,lagged_other_races,lagged_public_commute,lagged_income_per_capita,lagged_housing_units,lagged_vacant_housing_units,lagged_spatial_rent
0,36,047,005300,36047005300,53,Census Tract 53,G5020,S,1651843,5374846,+40.6708307,-074.0291182,"POLYGON ((-74.04894 40.66270, -74.04822 40.663...",36047005300,-241.530822,0.0,0.198901,0.287012,94.413245,199.051668,21926.838440,173.314816,-109.681906,138.060110,10968.75,0.0,0.031706,0.0,-10.0,-19.0,927.0,-12.0,16.0,11.0,0.000000,0.000000,0.0,2605.260,0.109861,-61.000000,49.000000,6500.0,-4.000000,1.000000,0.0
1,36,047,005900,36047005900,59,Census Tract 59,G5020,S,355163,0,+40.6794028,-074.0061193,"POLYGON ((-74.01170 40.67936, -74.01045 40.680...",36047005900,12033.214941,0.0,0.000000,-0.027196,187.867740,78.189595,822.062197,158.254830,-51.556596,151.221743,2605.26,0.0,0.109861,0.0,-61.0,49.0,6500.0,-4.0,1.0,14.0,0.284185,14.920378,0.0,1551.300,0.045701,70.333333,-33.666667,1131.0,7.333333,15.666667,0.0
2,36,047,004700,36047004700,47,Census Tract 47,G5020,S,283960,0,+40.6883662,-074.0018662,"POLYGON ((-74.00843 40.68602, -74.00816 40.686...",36047004700,-11522.486485,0.0,0.000000,-0.067625,170.264156,-173.384460,-9299.253349,214.356759,-11.264447,-11.195519,-4845.54,0.0,0.091227,0.0,-88.0,-7.0,2819.0,-17.0,30.0,-34.0,0.000000,0.000000,0.0,567.975,0.062015,124.000000,-13.000000,3073.5,23.500000,1.000000,0.0
3,36,047,008500,36047008500,85,Census Tract 85,G5020,S,243576,0,+40.6758570,-074.0064716,"POLYGON ((-74.01079 40.67683, -74.01017 40.677...",36047008500,-1710.671304,0.0,0.027318,0.108046,-897.245871,166.776227,3239.579385,12.914013,-55.333173,-189.172105,-1469.31,0.0,0.014170,0.0,309.0,-75.0,-353.0,51.0,1.0,-18.0,0.000000,0.000000,0.0,-1120.140,0.100544,-74.500000,21.000000,4659.5,-10.500000,15.500000,0.0


In [ ]:
mu=spatial_data_rh[['2022_2017_median_income','2022_2017_income_dist','2022_2017_edu_dist','2022_2017_other_races','2022_2017_commuters_by_public_transportation','2022_2017_income_per_capita','2022_2017_housing_units','2022_2017_vacant_housing_units','2022_2017_bachelors_degree','2022_2018_EP_POV','2022_2018_EP_MUNIT','2022_2018_EP_MOBILE','lagged_spatial_income','lagged_spatial_rent','lagged_income_dist']].mean()
sigma=mu=spatial_data_rh[['2022_2017_median_income','2022_2017_income_dist','2022_2017_edu_dist','2022_2017_other_races','2022_2017_commuters_by_public_transportation','2022_2017_income_per_capita','2022_2017_housing_units','2022_2017_vacant_housing_units','2022_2017_bachelors_degree','2022_2018_EP_POV','2022_2018_EP_MUNIT','2022_2018_EP_MOBILE','lagged_spatial_income','lagged_spatial_rent','lagged_income_dist']].std()

In [ ]:
spatial_data_rh_pred = spatial_data_rh[['geo_id',
                                        '2022_2017_median_income',
                                        '2022_2017_income_dist',
                                        '2022_2017_edu_dist',
                                        '2022_2017_other_races',
                                        '2022_2017_commuters_by_public_transportation',
                                        '2022_2017_income_per_capita',
                                        '2022_2017_housing_units',
                                        '2022_2017_vacant_housing_units',
                                        '2022_2017_bachelors_degree',
                                        '2022_2018_EP_POV',
                                        '2022_2018_EP_MUNIT',
                                        '2022_2018_EP_MOBILE',
                                        'lagged_spatial_income',
                                        'lagged_spatial_rent',
                                        'lagged_income_dist']]

In [ ]:
spatial_data_rh_pred

,geo_id,2022_2017_median_income,2022_2017_income_dist,2022_2017_edu_dist,2022_2017_other_races,2022_2017_commuters_by_public_transportation,2022_2017_income_per_capita,2022_2017_housing_units,2022_2017_vacant_housing_units,2022_2017_bachelors_degree,2022_2018_EP_POV,2022_2018_EP_MUNIT,2022_2018_EP_MOBILE,lagged_spatial_income,lagged_spatial_rent,lagged_income_dist
0,36047005300,-241.530822,0.198901,0.287012,94.413245,199.051668,21926.838440,173.314816,-109.681906,138.060110,0.000000,0.000000,0.0,2605.260,0.0,0.109861
1,36047005900,12033.214941,0.000000,-0.027196,187.867740,78.189595,822.062197,158.254830,-51.556596,151.221743,0.284185,14.920378,0.0,1551.300,0.0,0.045701
2,36047004700,-11522.486485,0.000000,-0.067625,170.264156,-173.384460,-9299.253349,214.356759,-11.264447,-11.195519,0.000000,0.000000,0.0,567.975,0.0,0.062015
3,36047008500,-1710.671304,0.027318,0.108046,-897.245871,166.776227,3239.579385,12.914013,-55.333173,-189.172105,0.000000,0.000000,0.0,-1120.140,0.0,0.100544


In [ ]:
spatial_data_rh_pred = spatial_data_rh_pred.fillna(0)
spatial_data_rh_pred.iloc[:,1:]=(spatial_data_rh_pred.iloc[:,1:]-mu)/sigma
spatial_data_rh_pred = spatial_data_rh_pred.fillna(0)
spatial_data_rh_pred['prediction'] = logreg.predict_proba(spatial_data_rh_pred.iloc[:,1:])[:, 1] 
spatial_data_rh_pred.head()

,geo_id,2022_2017_median_income,2022_2017_income_dist,2022_2017_edu_dist,2022_2017_other_races,2022_2017_commuters_by_public_transportation,2022_2017_income_per_capita,2022_2017_housing_units,2022_2017_vacant_housing_units,2022_2017_bachelors_degree,2022_2018_EP_POV,2022_2018_EP_MUNIT,2022_2018_EP_MOBILE,lagged_spatial_income,lagged_spatial_rent,lagged_income_dist,prediction
0,36047005300,-1.024999,1.076919,0.793511,-0.820375,0.180454,0.683693,0.974142,-3.714055,-0.131815,-1.0,-1.0,0.0,0.645157,0.0,2.587913,0.001008
1,36047005900,0.245487,-1.000000,-1.169942,-0.642574,-0.536305,-0.936876,0.802601,-2.275757,-0.049049,1.0,1.0,0.0,-0.020393,0.0,0.492537,0.012462
2,36047004700,-2.192624,-1.000000,-1.422581,-0.676066,-2.028238,-1.714060,1.441630,-1.278736,-1.070403,-1.0,-1.0,0.0,-0.641338,0.0,1.025345,0.000716
3,36047008500,-1.177061,-0.714749,-0.324830,-2.707045,-0.010952,-0.751243,-0.852903,-2.369207,-2.189600,-1.0,-1.0,0.0,-1.707341,0.0,2.283632,0.000714


In [ ]:
spatial_data_rh_pred.to_csv('redhook_gentrification_prediction.csv')